<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [1]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
!pip install bs4
!pip install tqdm
!pip install pandas
!pip install requests
!pip install hazm
!pip install unidecode
!pip install fasttext
!pip install scikit-learn
!pip install pandas
!pip install nltk
!pip install torch
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu

In [3]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import faiss
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [5]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [6]:
scraper = Scraper(current_year=1401, current_month=3)
scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [4]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [8]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [9]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [10]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [11]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای بعضی از مدل‌ها می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن‌ها بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [12]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset()

100%|██████████| 4000/4000 [00:25<00:00, 153.97it/s]


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٣. دریافت اخبار مرتبط با کوئری کاربر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این مرحله با استفاده از چهار مدل متفاوت زبانی، شروع به یادگیری روی خبرهای استخراج شده می‌کنیم و در انتها، نتیجه اجرای ده کوئری را بر روی هر یک مشاهده می‌کنیم. کوئری‌های مورد استفاده در این بخش در زیر آمده‌اند.
    </p>
</div>

In [13]:
QUERIES = [
    "نتیجه توافق برجام",
    "حمله ارتش روسیه به اوکراین",
    "افزایش نرخ تورم در کشور",
    "آمار فوتی‌های کرونا",
    "اقدامات شهرداری تهران",
    "صعود ایران به جام جهانی",
    "بیماری آبله میمونی",
    "جشنواره فیلم فجر",
    "رشد رمزارزها",
    "روش پخت غذا",
]

BOOLEAN_EXTRA_QUERIES = [
    ['اوپو', 'ردمی'],
    ['اوپو', 'ردمی', '-شیائومی'],
]

COMPLEX_QUERIES = [
    "چه رژیمی برای لاغری مناسب است؟",
]

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ١. مدل مبتنی بر روش boolean
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    برای ایجاد این مدل، از کلاس CountVectorizer کتابخانه sklearn استفاده می‌کنیم تا به ازای هر کلمه مشخص شود که در کدام داکیومنت‌ها آمده است. توجه کنید که امکان سرچ با استفاده از عملگرهای or و not نیز پیاده‌سازی شده است. کلمات موجود در یک زیرکوئری همگی رابطه‌ی and را دارند. یک لیست از زیرکوئری‌ها نقش or را ایفا می‌کند. در نهایت، برای exclude کردن یک عبارت از نتایج جست و جو، کافی است در ابتدای عبارت مد نظر کاراکتر "-" گذاشته شود تا به عنوان not عمل کند.
    </p>
</div>

In [14]:
class BooleanIR:

    def __init__(self):
        self.vectorizer = CountVectorizer(binary=True)
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def preprocess_query(self, query):
        preprocessor = Preprocessor(stopwords_path='stopwords.txt')
        preprocessed_query = []
        for subquery in query:
            if subquery[0] == '-':
                preprocessed_query.append('-' + (' '.join(preprocessor.preprocess(subquery[1:]))))
            else:
                preprocessed_query.append(' '.join(preprocessor.preprocess(subquery)))
        return preprocessed_query

    def predict(self, query, dataset, k):
        query = self.preprocess_query(query)
        query_result = []
        query_result_BUT = []
        for subquery in query:
            is_subquery_but = False
            if subquery[0] == '-':
                is_subquery_but = True
                subquery = subquery[1:]
            query_transform = self.vectorizer.transform([subquery]).todense().tolist()[0]
            query_indices = np.nonzero(query_transform)[0]
            for index, doc in self.dense_vectors_df.iterrows():
                if all(np.take(doc, query_indices)):
                    if is_subquery_but:
                        query_result_BUT.append(index)
                    else:
                        query_result.append(index)
        query_result = list(set(query_result) - set(query_result_BUT))
        k = min(k, len(query_result))
        return dataset.iloc[query_result[:k]]

    def save_boolean_model(self, path='BooleanIR_model.pickle'):
        with open(path, "wb") as file:
            pickle.dump(self, file)

    def load_boolean_model(self, path='BooleanIR_model.pickle'):
        with open(path, "rb") as file:
            return pickle.load(file)

    def prepare(self, dataset, mode, save=False):
        model = None
        if mode == 'train':
            self.fit_transform_vectorizer(dataset)
            model = self
        if mode == 'load':
            model = self.load_boolean_model()
        if save:
            self.save_boolean_model()
        return model

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل BooleanIR ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [15]:
booleanIR_model = BooleanIR().prepare(mini_preprocessed_texts, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های مشخص شده را مشاهده می‌کنیم.
    </p>
</div>

In [16]:
for i, query in enumerate(QUERIES):
    print(f"Query {i + 1}: {query}")
    display(booleanIR_model.predict([query], mini_dataset, k=10))

Query 1: نتیجه توافق برجام


,date,title,intro,body,category
1,پنجشنبه ۳۰ اردیبهشت ۱۴۰۰ - ۱۵:۰۶,آژانس بین‌المللی انرژی اتمی: در حال رایزنی با ...,آژانس بین‌المللی انرژی اتمی در بیانیه‌ای اعلام...,به گزارش همشهری‌آنلاین به نقل از فارس، آژانس ...,Politics
133,شنبه ۱۱ دی ۱۴۰۰ - ۰۷:۳۵,مذاکرات مثبت است اما ... | یک خواسته ایران کار...,یک رسانه آلمانی می‌گوید فضای کلی مذاکرات اتمی ...,به گزارش همشهری‌آنلاین به نقل از شفقنا، دپ‌آ د...,Politics
389,شنبه ۱۱ دی ۱۴۰۰ - ۰۷:۳۲,گزینه‌های سپاه روی میز وین! | بهترین پیشنهاد د...,علی‌رغم فضاسازی‌های صهیونیست‌ها و رسانه‌های غر...,به گزارش همشهری آنلاین، روزنامه کیهان در یاددا...,Politics
265,دوشنبه ۱۸ بهمن ۱۴۰۰ - ۱۸:۵۳,رئیسی در دیدار مقام اروپایی: در صورت لغو تحریم...,رئیسی با اشاره به بدعهدی آمریکایی‌ها در برجام،...,به گزارش همشهری‌آنلاین به نقل از پایگاه اطلاع ...,Politics
460,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۶:۱۰,منتظر جنگ جهانی سوم باشیم؟ | امکان چریکی شدن ج...,جهانگیر کرمی، عضو هیات علمی دانشگاه تهران و کا...,همشهری‌آنلاین - اصغر صوفی: درگیری میان اوکراین...,World
344,شنبه ۴ دی ۱۴۰۰ - ۰۷:۴۱,خبر مهم منبع اروپایی از مذاکرات وین | احتمال ت...,یک منبع اروپایی گفت: بین ما و هیات ایرانی نهای...,به گزارش همشهری آنلاین به نقل از ایرنا، ایران ...,Politics
284,شنبه ۲۸ اسفند ۱۴۰۰ - ۱۴:۳۵,آغاز سال ۱۴۰۱ با توافق در وین؟ | ادعای ایندیپن...,یک رسانه‌ انگلیسی به نقل از یکی از طرف‌های مذا...,به گزارش همشهری آنلاین به نقل از فارس، روزنامه...,Politics
353,چهارشنبه ۱ اردیبهشت ۱۴۰۰ - ۱۳:۱۵,احمدی‌نژاد بعد از قرارداد چین به مذاکرات وین ح...,محمود احمدی‌نژاد جزو اولین کسانی بود که به قرا...,همشهری‌آنلاین - محمد بیاتی: جنجال‌سازی‌های محم...,Politics
290,چهارشنبه ۲ تیر ۱۴۰۰ - ۲۲:۲۲,پمپئو: اگر قوی نباشیم ایران ما را تنبیه می‌کند,وزیر امور خارجه سابق۴ آمریکا در بخشی از یادداش...,به گزارش همشهری‌آنلاین به نقل از فارس، «مایک پ...,Politics
35,پنجشنبه ۸ مهر ۱۴۰۰ - ۱۵:۱۶,موضع ایران در برابر طالبان چیست؟ | خطیب‌زاده: ...,​ سخنگوی وزارت امور خارجه با تاکیدبراینکه اروپ...,به گزارش همشهری‌آنلاین به نقل از اداره کل اطل...,Politics


Query 2: حمله ارتش روسیه به اوکراین


,date,title,intro,body,category
578,جمعه ۶ اسفند ۱۴۰۰ - ۱۶:۰۷,پوتین: آماده مذاکره با اوکراین هستیم | موضع ری...,رئیس جمهور چین در یک تماس تلفنی با رئیس جمهور ...,به گزارش همشهری‌آنلاین به نقل از فارس، رسانه‌ه...,World
771,چهارشنبه ۲۵ اسفند ۱۴۰۰ - ۱۹:۲۶,ادعای ناتو درباره شرایط نیروهای روسیه | به ارس...,مقامات ارشد نظامی ناتو مدعی شده‌اند که ولادیمی...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World
518,یکشنبه ۷ فروردین ۱۴۰۱ - ۰۸:۵۹,ببینید | حمله موشک‌های «کالیبر» روسیه به انبار...,وزارت دفاع روسیه می‌گوید ارتش این کشور بامداد ...,به گزارش همشهری آنلاین، وزارت دفاع روسیه تصریح...,World
778,پنجشنبه ۵ اسفند ۱۴۰۰ - ۲۰:۴۸,ارتش روسیه در حال پیشروی به سمت کی‌یف است | آپ...,یکی از مقام‌های ارشد دفاعی آمریکا می‌گوید، حمل...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,World
3594,یکشنبه ۱ اسفند ۱۴۰۰ - ۱۱:۴۵,حمله به اینترنت؛ نخستین هدف ارتش روسیه در اوکراین,اتفاقات رخ داده در قزاقستان تا حدودی می‌تواند ...,به گزارش همشهری آنلاین و به نقل از روزنامه نیو...,IT
595,یکشنبه ۲۴ بهمن ۱۴۰۰ - ۲۳:۵۱,بحران اوکراین | محموله موشک‌های استینگر به کی‌...,وزارت دفاع اوکراین اعلام کرد، این کشور روز یکش...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,World
468,چهارشنبه ۳ فروردین ۱۴۰۱ - ۰۱:۳۰,دبیرکل سازمان ملل خطاب به روسیه: به «جنگ پوچ» ...,آنتونیو گوترش، دبیرکل سازمان ملل از روسیه خواس...,به گزارش همشهری آنلاین به نقل از خبرگزاری فران...,World
725,شنبه ۳ اردیبهشت ۱۴۰۱ - ۱۶:۰۱,روسیه تلفات انسانی رزمناو غرق شده را اعلام کرد,یک هفته پس از غرق شدن رزمناو مسکووا، وزارت دفا...,به گزارش همشهری آنلاین به نقل از ایسنا، یک هفت...,World
728,جمعه ۶ خرداد ۱۴۰۱ - ۱۸:۲۸,کنترل یک شهر استراتژیک اوکراین در دست روسیه | ...,مقامات اوکراینی امروز (جمعه) اعلام کردند که نی...,به گزارش همشهری آنلاین، اولکسی آرستویچ، مشاور ...,World
3547,جمعه ۶ اسفند ۱۴۰۰ - ۱۹:۲۱,نبرد سایبری ادامه دارد | حمله سنگین هکرهای بلا...,مقامات اوکراینی روز جمعه ادعا کردند، هکرهایی ک...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,IT


Query 3: افزایش نرخ تورم در کشور


...

,date,title,intro,body,category
2826,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
3083,شنبه ۶ آذر ۱۴۰۰ - ۱۲:۵۲,قصر شیرین به تلویزیون می‌آید,منتخبی از فیلم‌های سیدرضا میرکریمی شامل «زیر ن...,به گزارش همشهری آنلاین به نقل از پایگاه اطلاع ...,Culture
3978,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۰۰:۰۶,عکس | ژست متفاوت صبا سهیلی کنار سینا مهراد در ...,تصاویری از سینا مهراد و خواهرش صبا سهیلی را مش...,به گزارش همشهری آنلاین، در دومین روز برگزاری چ...,LifeSkills
2957,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۶:۵۲,«شهربانو» بهترین فیلم جشنواره سیدنی شد,فیلم سینمایی «شهربانو» ساخته مریم بحرالعلومی ب...,به گزارش همشهری آنلاین، فیلم سینمایی «شهربانو»...,Culture
2834,دوشنبه ۲۷ دی ۱۴۰۰ - ۱۲:۱۰,مریلا زارعی و بهروز شعیبی با هناس به جشنواره م...,نام فیلم سینمایی «صورت فلکی» به «هِناس» تغییر ...,به گزارش همشهری آنلاین به نقل از روابط عمومی پ...,Culture
2841,جمعه ۱۳ فروردین ۱۴۰۰ - ۱۲:۳۳,خورشید مجیدی در اسپانیا روی پرده می‌رود,فیلم سینمایی «خورشید» به کارگردانی مجید مجیدی ...,به گزارش همشهری آنلاین، کمتر از ۴۵ روز مانده ب...,Culture
2972,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۶:۳۹,زندگینامه: ساره بیات (۱۳۵۸-),ساره بیات در سال ۱۳۵۸ در مشهد بدنیا آمد.,به گزارش همشهری آنلاین، ساره بیات، زاده ۱۴ مه...,Culture
2976,شنبه ۹ بهمن ۱۴۰۰ - ۱۴:۳۴,نخستین عکس از رعنا آزادی‌ور در فیلم مرد بازنده,رعنا آزادی‌ور در دومین همکاری با محمدحسین مهدو...,به گزارش همشهری‌آنلاین به نقل از مهر، فیلمنامه...,Culture
3104,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۲:۵۷,لحظه شماری می‌کردیم رهبر انقلاب این فیلم را بب...,تهیه‌کننده فیلم «موقعیت مهدی» از نگاه حکیمانه ...,به گزارش همشهری‌آنلاین به نقل از تسنیم، پیمان ...,Culture
3108,یکشنبه ۸ فروردین ۱۴۰۰ - ۰۸:۱۷,زندگینامه: هانیه توسلی (۱۳۵۸-),هانیه توسلی سال ۱۳۵۸ در همدان به دنیا آمد.,به گزارش همشهری آنلاین، هانیه توسلی زادهٔ ۱۴ ...,Culture


Query 9: رشد رمزارزها


,date,title,intro,body,category
3328,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۰:۵۷,بررسی ۵ صرافی آنلاین ارز دیجیتال ایرانی,اولین قدم برای شروع معاملات ارزهای دیجیتال، ان...,این افزایش تعداد صرافی‌های ارز دیجیتال، ممکن ا...,IT
898,پنجشنبه ۱۵ مهر ۱۴۰۰ - ۰۹:۵۶,قیمت بیت‌کوین از ۵۵ هزار دلار عبور کرد,قیمت بیت‌کوین، مهم‌ترین رمزارز جهان، به روند ر...,به گزارش همشهری آنلاین، مهر به نقل از سی‌ان‌بی...,Economy
3333,پنجشنبه ۲۹ اردیبهشت ۱۴۰۱ - ۰۸:۴۱,گردهمایی اقتصادی با محوریت بیت‌کوین | السالواد...,السالوادور، اولین کشوری که بیت‌کوین را به‌عنوا...,به‌گزارش همشهری آنلاین به نقل از کوین‌تلگراف، ...,IT
3975,دوشنبه ۱ شهریور ۱۴۰۰ - ۰۸:۱۳,جهش قیمت بیت‌کوین,قیمت بیت‌کوین، مهمترین رمزارز جهان، در معاملات...,به گزارش همشهری آنلاین، مهر به نقل از سی‌ان‌بی...,LifeSkills
3593,یکشنبه ۱۵ اسفند ۱۴۰۰ - ۱۲:۰۲,سرمایه‌گذاری در بورس بهتر است یا ارز دیجیتال ؟,بازار بورس و ارزهای دیجیتال، دو مورد از محبوب‌...,بازار ارزهای دیجیتال به طور قابل توجهی بی‌ثبات...,IT
1034,دوشنبه ۲۱ تیر ۱۴۰۰ - ۱۱:۲۸,ارزش ارزهای دیجیتال بالاخره صعودی شد,قیمت انواع ارزهای شاخص دیجیتال امروز در روند م...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Economy
3211,شنبه ۲۴ اردیبهشت ۱۴۰۱ - ۰۹:۵۹,دولت برای رمزارزها چارچوب حداقلی تعیین کند,مدیرعامل نوبیتکس با بیان اینکه در مورد قانونگذ...,اگر دولت در این حد وارد حوزه رمزارزها می‌شد که...,IT
3345,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۸:۴۴,چطور با آیفون در ارز دیجیتال سرمایه‌گذاری کنیم؟,با رشدی که بازار ارزهای دیجیتال در سال ۲۰۲۱ تج...,ارز دیجیتال چیست؟\nارز دیجیتال در اصل یک نوع پ...,IT
918,یکشنبه ۱۴ فروردین ۱۴۰۱ - ۱۲:۳۳,پیش بینی وضعیت بازار سرمایه در سال ۱۴۰۱ ‌| رشد...,یک کارشناس بازار سرمایه ضمن ارزیابی عوامل تاثی...,به گزارش همشهری آنلاین به نقل از ایرنا، معاملا...,Economy
3351,یکشنبه ۱۷ بهمن ۱۴۰۰ - ۱۷:۲۳,چند رمزارز خوب برای سرمایه‌گذاری در سال ۲۰۲۲,سال ۲۰۲۱ نیز با تمام فرازو نشیب‌هایی که در دنی...,پس از مدتی که بیت کوین نفس تازه کرد، مجدداً شر...,IT


Query 10: روش پخت غذا


,date,title,intro,body,category
3973,سه‌شنبه ۲۲ تیر ۱۴۰۰ - ۱۶:۳۰,طرز تهیه هویج پلو | فوت و فن درست کردن این غذا...,هویج ‌پلو از جمله پلوهای مخلوط قدیمی است که مع...,به گزارش همشهری آنلاین - صدیقه کیان منش‌راد: ه...,LifeSkills
3659,سه‌شنبه ۴ خرداد ۱۴۰۰ - ۱۴:۳۰,طرز تهیه‌ بریانی | نکته‌های کلیدی پخت غذای محل...,بریونی یا بریانی یکی از چندین غذای لذیذ سنتی و...,به گزارش همشهری آنلاین، شاید اسم غذاهای محلی ا...,LifeSkills
3853,پنجشنبه ۶ خرداد ۱۴۰۰ - ۰۷:۳۲,این روغن‌ها نباید مصرف شوند | روغن‌های مناسب آ...,یک متخصص تغذیه مطرح کرد: مصرف روغن‌های حیوانی ...,به گزارش همشهری آنلاین، دکتر سارا صافی در گفت‌...,LifeSkills
3730,پنجشنبه ۳۱ تیر ۱۴۰۰ - ۰۸:۴۰,طرز تهیه کباب تابه‌ای | به ۲ شیوه ساده کباب دی...,۲ طرز تهیه ساده کباب تابه‌ای زعفرانی آبدار مجل...,به گزارش همشهری آنلاین به نقل از اکالا، تابه‌ا...,LifeSkills
3604,چهارشنبه ۲۹ اردیبهشت ۱۴۰۰ - ۱۸:۳۴,طرز تهیه ماکارونی با سویا ؛ همراه با یک دستور ...,ماکارونی یک غذای ویژه است که علاقه‌مندان زیادی...,به گزارش همشهری آنلاین، ماکارونی یک غذای ویژه ...,LifeSkills
3861,دوشنبه ۱۱ بهمن ۱۴۰۰ - ۰۷:۳۷,دانستنی‌هایی درباره برنج که باورتان نمی‌شود | ...,برنج یک خوراک محبوب جهانی است اما چالش‌های خود...,همشهری آنلاین - فاطمه عباسی: برنج قوت غالب مرد...,LifeSkills
3865,چهارشنبه ۵ خرداد ۱۴۰۰ - ۱۷:۳۲,طرز تهیه ماهی با روش شمالی | نکات طلایی برای ل...,ماهی انواع و اقسام مختلف با شیوه پخت متفاوتی د...,به گزارش همشهری آنلاین، پزشکان می‌گویند ماهی ب...,LifeSkills
3803,شنبه ۳۰ مرداد ۱۴۰۰ - ۱۵:۵۰,راهکارهای گرفتن تلخی قیمه و پیشگیری از تلخ شدن...,احتمالاً برای شما هم پیش آمده است که خورشت یا ...,به گزارش همشهری آنلاین به نقل از تابناک، مزه ت...,LifeSkills
2910,پنجشنبه ۲۱ مرداد ۱۴۰۰ - ۰۷:۵۴,دستور طبخ غذا برای دیکتاتورها | اگر از دریچه آ...,روزنامه‌نگاری که خود زمانی آشپزی را تجربه کرده...,به گزارش همشهری آنلاین، کتاب «مصائب آشپزی برای...,Culture
3744,سه‌شنبه ۲۵ آبان ۱۴۰۰ - ۰۶:۱۳,طرز تهیه چیکن آلفردو | ترفند تبدیل این غذا به ...,پاستای فوری چیکن آلفردو یک انتخاب بسیار ساده و...,مواد لازم برای درست کردن چیکن آلفردو \n۵۰۰ گرم...,LifeSkills


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های ترکیبی با منطق or و not را مشاهده می‌کنیم.
    </p>
</div>

In [17]:
for query in BOOLEAN_EXTRA_QUERIES:
    print(f"Extra Query: {query}")
    display(booleanIR_model.predict(query, mini_dataset, k=10))

Extra Query: ['اوپو', 'ردمی']


,date,title,intro,body,category
3520,چهارشنبه ۱۵ دی ۱۴۰۰ - ۱۸:۳۷,ردمی K50 شیائومی؛ ماه آینده به بازار می‌آید | ...,شرکت شیائومی امروز تایید کرد که گوشی «ردمی کی...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3393,سه‌شنبه ۵ بهمن ۱۴۰۰ - ۱۴:۴۲,پوکو X4 با مشخصاتی ایده‌آل در راه است,یک گوشی هوشمند پوکو (POCO) با شماره مدل ۲۲۰۱۱۱...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3522,جمعه ۵ فروردین ۱۴۰۱ - ۱۲:۳۸,تصاویر | قاب کولردار گوشی پرچم‌دار اوپو به باز...,«اوپو» یک ماه پیش گوشی Find X5 Pro (فایند ایکس...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3589,جمعه ۲۴ دی ۱۴۰۰ - ۱۱:۴۰,نخستین تبلت اوپو به بازار می‌آید | قدرتمند و ه...,ساخت و ارائه تبلت اوپو که در نیمه دوم سال ۲۰۲۱...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3525,یکشنبه ۲۶ دی ۱۴۰۰ - ۲۲:۱۶,شیائومی سرمایه‌گذاری بیشتری بر روی فناوری باند...,آژانس ثبت کسب و کار دولتی چین (QCC)، اخیرا پای...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3336,سه‌شنبه ۲۶ بهمن ۱۴۰۰ - ۱۷:۲۴,خبری از تغییر نیست | عرضه ردمی ۱۰ مدل ۲۰۲۲ با ...,شیائومی سال گذشته ردمی ۱۰ (Redmi 10) خود را به...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3401,شنبه ۲۰ فروردین ۱۴۰۱ - ۱۴:۴۴,سازنده عجله‌ای ندارد | تاشوی وان‌پلاس؛ شاید وق...,به نظر می‌رسد که در سال جاری میلادی شاهد ورود ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3221,چهارشنبه ۲۹ دی ۱۴۰۰ - ۰۹:۵۹,مشخصات نخستین تبلت ویوو فاش شد,پس از شایعات فراوان، معاون اجرایی ویوو (Vivo) ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3353,شنبه ۳۱ اردیبهشت ۱۴۰۱ - ۱۰:۴۲,معرفی بهترین گوشی تا قیمت ۸ میلیون تومان,شما می توانید با بررسی معروف ترین و بهترین گوش...,از سوی دیگر در سال های اخیر قیمت مربوط به گوشی...,IT
3427,دوشنبه ۲۷ دی ۱۴۰۰ - ۰۹:۵۷,فاش شدن مشخصات گوشی جدید اوپو | رقیبی جدی برای...,در حالی که انتظار می‌رود، گوشی هوشمند «اوپو فا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT


Extra Query: ['اوپو', 'ردمی', '-شیائومی']


,date,title,intro,body,category
3522,جمعه ۵ فروردین ۱۴۰۱ - ۱۲:۳۸,تصاویر | قاب کولردار گوشی پرچم‌دار اوپو به باز...,«اوپو» یک ماه پیش گوشی Find X5 Pro (فایند ایکس...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3427,دوشنبه ۲۷ دی ۱۴۰۰ - ۰۹:۵۷,فاش شدن مشخصات گوشی جدید اوپو | رقیبی جدی برای...,در حالی که انتظار می‌رود، گوشی هوشمند «اوپو فا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3589,جمعه ۲۴ دی ۱۴۰۰ - ۱۱:۴۰,نخستین تبلت اوپو به بازار می‌آید | قدرتمند و ه...,ساخت و ارائه تبلت اوپو که در نیمه دوم سال ۲۰۲۱...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3401,شنبه ۲۰ فروردین ۱۴۰۱ - ۱۴:۴۴,سازنده عجله‌ای ندارد | تاشوی وان‌پلاس؛ شاید وق...,به نظر می‌رسد که در سال جاری میلادی شاهد ورود ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3317,سه‌شنبه ۲۸ دی ۱۴۰۰ - ۰۹:۳۷,مشخصات جدیدترین گوشی هایسنس لو رفت,گوشی هوشمند جدید هایسنس با شماره مدل HNR۵۵۳T گ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3321,شنبه ۲ بهمن ۱۴۰۰ - ۱۱:۴۸,وان‌پلاس در اندیشه ارائه یک گوشی جدید در ۳ ماه...,وان‌پلاس در حال حاضر جدیدترین گوشی هوشمند پرچم...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3263,پنجشنبه ۲۵ فروردین ۱۴۰۱ - ۱۲:۴۵,کدام گوشی‌های اندرویدی بیشترین تشعشع را دارند؟,مدل‌های قدیمی پیکسل، وان پلاس و سونی ایکس پریا...,به گزارش همشهری آنلاین به نقل از ایسنا، بر هیچ...,IT


<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ۲. مدل مبتنی بر روش TF-IDF
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش از کلاس TfidfVectorizer از کتاب‌خانه‌ی sklearn استفاده می‌کنیم. پس از ساخته شدن یک نمونه از این کلاس، می‌توانیم متد prepare آن را صدا کنیم و با این کار، با توجه به پارامترهایی که به آن می‌دهیم، مدل را به یادگیری واداریم یا مدلی که پیش‌تر ذخیره کرده‌ایم را بازیابی کنیم. عملیات یادگیری با استفاده از متد fit_transform_vectorizer
        رخ می‌دهد و در این حین، بردارهای نتیجه و همین‌طور کلمات مربوط به آن‌ها هم ذخیره می‌شوند.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        علاوه بر این، می‌توانیم با استفاده از متدهای save_TF_IDF_model و load_TF_IDF_model این مدل و جزئیاتش را ذخیره یا بازیابی کنیم. البته اگر بخواهیم عمل ذخیره را بلافاصله پس از پایان یادگیری انجام دهیم هم می‌توانیم مقدار True را به آرگومان save در متد prepare بدهیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از پایان یادگیری یا بازیابی مدل یادگرفته شده از فایل ذخیره شده، می‌توانیم با استفاده از متد predict به تعداد دلخواه خبر مرتبط به درخواستمان را به دست بیاوریم. برای این کار، مدلمان بردارهای مربوط به درخواست را به دست می‌آورد و با محاسبه‌ی کسینوس‌ها و به دست آوردن میزان شباهت بین درخواست و خبرهای موجود، شبیه‌ترین خبرها را خروجی می‌دهد.
    </p>
</div>

In [14]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query = ' '.join(Preprocessor(stopwords_path='stopwords.txt').preprocess(query))
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        dense_vectors = self.dense_vectors_df.values.tolist()
        df_cosine_sim = list(map(lambda doc: self.cosine_sim(query_transform, doc), dense_vectors))
        self.dense_vectors_df['query_sim'] = df_cosine_sim
        indices = self.dense_vectors_df.nlargest(k, 'query_sim').index
        self.dense_vectors_df = self.dense_vectors_df.drop(columns=['query_sim'])
        return dataset.iloc[indices]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self, file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

    def prepare(self, dataset, mode, save=False):
        model = None
        if mode == 'train':
            self.fit_transform_vectorizer(dataset)
            model = self
        if mode == 'load':
            model = self.load_TF_IDF_model()
        if save:
            self.save_TF_IDF_model()
        return model

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل TF-IDF ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [15]:
TF_IDF_model = TF_IDF().prepare(mini_preprocessed_texts, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های مشخص شده را مشاهده می‌کنیم.
    </p>
</div>

In [ ]:
for i, query in enumerate(QUERIES):
    print(f"Query {i + 1}: {query}")
    display(TF_IDF_model.predict(query, mini_dataset, k=10))

Query 1: نتیجه توافق برجام


,date,title,intro,body,category
282,سه‌شنبه ۱۷ فروردین ۱۴۰۰ - ۲۳:۴۱,جهانگیری: فراموش نکرده‌ایم عده‌ای در مجلس برجا...,جهانگیری، معاون اول رئیس‌جمهوری، گفت: فراموش ن...,به گزارش همشهری‌آنلاین به نقل از انتخاب، جهانگ...,Politics
185,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۰۷:۳۰,درخواست مهم گوترش از آمریکا درباره ایران | از ...,دبیرکل سازمان ملل متحد در یک گزارش رسمی از آمر...,به گزارش همشهری آنلاین به نقل از فارس، «آنتونی...,Politics
284,شنبه ۲۸ اسفند ۱۴۰۰ - ۱۴:۳۵,آغاز سال ۱۴۰۱ با توافق در وین؟ | ادعای ایندیپن...,یک رسانه‌ انگلیسی به نقل از یکی از طرف‌های مذا...,به گزارش همشهری آنلاین به نقل از فارس، روزنامه...,Politics
214,دوشنبه ۱۰ آبان ۱۴۰۰ - ۱۴:۵۱,چین: آمریکا باید سیاست اشتباه «فشار حداکثری» ع...,سخنگوی وزارت امور خارجه چین با تأکید بر حمایت ...,به گزارش همشهری‌آنلاین به نقل از فارس، «وانگ و...,Politics
398,یکشنبه ۱۴ شهریور ۱۴۰۰ - ۱۲:۰۸,نماینده تهران در مجلس: ما از مذاکره نمی‌ترسیم ...,نماینده تهران در مجلس گفت: در مجلس نهم مخالفان...,به گزارش همشهری آنلاین به نقل از برنا، اسماعیل...,Politics
19,پنجشنبه ۱۸ آذر ۱۴۰۰ - ۱۶:۲۷,بررسی متون پیشنهادی ایران در مذاکرات امروز وین...,نشست کمیسیون مشترک برجام با محور بررسی متون پی...,به گزارش همشهری آنلاین، نشست کمیسیون مشترک برج...,Politics
77,چهارشنبه ۹ تیر ۱۴۰۰ - ۱۹:۱۷,آغاز نشست شورای امنیت سازمان ملل درباره برجام,نشست شورای امنیت سازمان ملل امروز چهارشنبه برا...,به گزارش همشهری آنلاین به نقل از ایرنا، رز ما...,Politics
69,دوشنبه ۲۵ مرداد ۱۴۰۰ - ۲۱:۲۶,آژانس اتمی: ایران ۲۰۰ گرم اورانیوم فلزی ۲۰ درص...,آژانس بین‌المللی انرژی اتمی در گزارشی جدید ادع...,به گزارش همشهری‌آنلاین به نقل از فارس، آژانس ب...,Politics
208,چهارشنبه ۲۰ بهمن ۱۴۰۰ - ۱۶:۵۵,مانع اصلی دولت بایدن برای بازگشت به برجام | جا...,معاون سخنگوی‌ وزارت امور خارجه آمریکا گفت واشن...,به گزارش همشهری‌آنلاین به نقل از فارس، «جالینا...,Politics
170,یکشنبه ۱۰ مرداد ۱۴۰۰ - ۱۳:۲۴,توصیه ظریف به وزیر خارجه دولت رئیسی | واردات چ...,وزیر امور خارجه با بیان اینکه «من شهادت می‌دهم...,به گزارش همشهری آنلاین به نقل از مهر، محمدجوا...,Politics


Query 2: حمله ارتش روسیه به اوکراین


,date,title,intro,body,category
733,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۰:۲۸,ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سو...,رئیس جمهوری روسیه صبح امروز اعلام کرد که در پا...,به گزارش همشهری آنلاین به نقل از تسنیم، شورای ...,World
489,جمعه ۵ فروردین ۱۴۰۱ - ۲۳:۳۷,کشته شدن یک فرمانده روسی دیگر در خرسون | آمار ...,ارتش اوکراین ادعا کرد، ژنرال یاکوف رزانتسف از ...,به گزارش همشهری آنلاین، ایرنا به نقل از سی ان ...,World
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
528,شنبه ۷ اسفند ۱۴۰۰ - ۱۰:۳۴,ارتش روسیه شهر ملیتوپل را به تصرف خود درآورد,وزارت دفاع روسیه دقایقی پیش اعلام کرد، نیروهای...,به گزارش همشهری آنلاین و به نقل از خبرگزاری ای...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
444,دوشنبه ۲۵ بهمن ۱۴۰۰ - ۰۸:۲۴,روسیه ژست حمله به اوکراین را گرفت | شاید چین ه...,یک استاد ژئوپلیتیک دانشگاه با اشاره به اینکه ر...,به گزلرش همشهری آنلاین، عبدالرضا فرجی راد دربا...,World
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
571,یکشنبه ۲۴ بهمن ۱۴۰۰ - ۱۱:۲۹,حمله قریب‌الوقوع روسیه به اوکراین؟,یک مقام کاخ الیزه می‌گوید در تماس تلفنی رئیس ج...,به گزارش همشهری آنلاین به نقل از فارس، یک مقام...,World
542,پنجشنبه ۱۲ اسفند ۱۴۰۰ - ۱۸:۰۱,جنگ خانه به خانه در ماریوپل | ارتش اوکراین: شه...,از شواهد برمی‌آید که نیروهای ارتش روسیه رفته‌ر...,به گزارش همشهری آنلاین، سخنگوی گارد ملی اوکرای...,World


Query 3: افزایش نرخ تورم در کشور


...

,date,title,intro,body,category
3013,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۴:۴۵,جشنواره چهلم فیلم فجر فرش قرمز ندارد,یزدان عشیری از تغییرات فضای بصری و دکور جشنوار...,به گزارش همشهری آنلاین، یزدان عشیری مدیر روابط...,Culture
2826,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
2940,چهارشنبه ۲۱ مهر ۱۴۰۰ - ۱۶:۰۸,تئاتر فجر ۴۰ قطعاً آثار خارجی خواهد داشت | حضو...,دبیر چهلمین جشنواره تئاتر فجر با تأکید بر اینک...,به گزارش همشهری آنلاین به نقل از مهر، حسین مسا...,Culture
3184,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۱۵:۴۰,بی‌مادر در مسیر جشنواره فیلم فجر,فیلم سینمایی «بی‌مادر» این روزها در حال آماده‌...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Culture
3145,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۱۵:۵۶,درخشش کلاس رانندگی در آلمان,مرضیه ریاحی از هجدهمین جشنواره جهانی فیلم کوتا...,به گزارش همشهری آنلاین به نقل از روابط عمومی ف...,Culture
3113,سه‌شنبه ۹ شهریور ۱۴۰۰ - ۱۶:۰۸,جشنواره فیلم کودک و نوجوان به زودی برگزار می‌شود,صاحبان ۵۲۴ فیلم ایرانی و خارجی متقاضی حضور در ...,به گزارش همشهری آنلاین به نقل از ستاد اطلاع‌رس...,Culture
2941,یکشنبه ۱۵ اسفند ۱۴۰۰ - ۱۰:۱۳,روایت پسر داریوش ارجمند از آهنگسازی برای فیلم‌...,امیریل ارجمند درباره فعالیت‌های هنری نظرات خود...,به گزارش همشهری آنلاین به نقل از صبا، امیریل ا...,Culture
3103,شنبه ۱۱ دی ۱۴۰۰ - ۱۱:۵۰,چتر حمایت از تئاتری‌ها گسترده شد,وزیر فرهنگ و ارشاد اسلامی شامگاه جمعه دهم دی ب...,به گزارش همشهری آنلاین به نقل از روابط عمومی و...,Culture
3003,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۰:۴۵,حاشیه‌های عجیب اختتامیه جشنواره فیلم فجر ۱۴۰۰ ...,چهلمین جشنواره فیلم فجر که با ماجراهای زیادی ه...,به گزارش همشهری آنلاین به نقل از ایسنا، آیین ا...,Culture
3026,چهارشنبه ۶ بهمن ۱۴۰۰ - ۱۳:۵۷,«مرد بازنده» برنده شد | آخرین خبرها از پیش فرو...,در دو ساعت اولیه فروش آنلاین بلیت‌های جشنواره ...,به گزارش همشهری آنلاین به نقل از ایرنا، فروش ب...,Culture


Query 9: رشد رمزارزها


,date,title,intro,body,category
1195,جمعه ۱۴ خرداد ۱۴۰۰ - ۱۵:۲۱,برخورد قهری با حوزه رمزارزها، منجربه خروج سرما...,به گفته دبیر کمیسیون رمزارز و بلاک‌چین سازمان ...,به گزارش همشهری آنلاین به نقل از ایسنا، علی با...,Economy
969,یکشنبه ۲۳ خرداد ۱۴۰۰ - ۱۹:۳۱,بخش خصوصی برای رمزارزها برق تولید می‌کند؟,هر چند هنوز در ایران رمزارزها متولی مشخصی ندار...,به گزارش همشهری آنلاین به نقل از ایسنا، از ابت...,Economy
3259,پنجشنبه ۱۹ اسفند ۱۴۰۰ - ۱۱:۵۹,پرده‌برداری از ۱۰راز بازار رمزارزها در ایران,به گزارش خبرگزاری عصرایران، بازار رمزارزها دیگ...,به گزارش خبرگزاری عصرایران، بازار رمزارزها دیگ...,IT
3345,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۸:۴۴,چطور با آیفون در ارز دیجیتال سرمایه‌گذاری کنیم؟,با رشدی که بازار ارزهای دیجیتال در سال ۲۰۲۱ تج...,ارز دیجیتال چیست؟\nارز دیجیتال در اصل یک نوع پ...,IT
3211,شنبه ۲۴ اردیبهشت ۱۴۰۱ - ۰۹:۵۹,دولت برای رمزارزها چارچوب حداقلی تعیین کند,مدیرعامل نوبیتکس با بیان اینکه در مورد قانونگذ...,اگر دولت در این حد وارد حوزه رمزارزها می‌شد که...,IT
962,سه‌شنبه ۲۱ اردیبهشت ۱۴۰۰ - ۱۰:۳۴,رمز ارزهایی که نباید بخرید,تبلیغ فروش رمزارزها با قیمتی کمتر از بازار در ...,به گزارش همشهری آنلاین به نقل از ایسنا، بازار ...,Economy
988,شنبه ۲۴ مهر ۱۴۰۰ - ۱۰:۴۳,بیتکوین ۶۱ هزار دلاری شد,خبر احتمال صدور مجوز برای فعالیت اولین صندوق E...,به گزارش همشهری آنلاین، ایسنا به نقل از سی‌ان‌...,Economy
3364,چهارشنبه ۲۱ اردیبهشت ۱۴۰۱ - ۱۷:۴۶,شیبا اینو با وجود ضرر همچنان محبوب است | رمزار...,بازار رمزارزها در هفته گذشته دستخوش تغییراتی ش...,به‌ گزارش همشهری آنلاین به نقل از واچر نیوز، ر...,IT
970,چهارشنبه ۲۲ اردیبهشت ۱۴۰۰ - ۰۷:۴۸,آیا وارد دور تازه‌ای از ریزش بیت‌کوین می‌شویم؟,ناتوانی بیت‌کوین در فتح کانال ۶۰ هزار دلاری با...,به گزارش همشهری‌آنلاین، ایسنا به نقل از سی‌ان‌...,Economy
714,سه‌شنبه ۱ تیر ۱۴۰۰ - ۰۸:۱۴,قیمت بیت‌کوین در شرق تعیین می‌شود,محدودیت‌ها و ممنوعیت‌های ایجادشده از سوی دولت ...,همشهری - ساسان شادمان‌منفرد: چین با ممنوع کردن...,World


Query 10: روش پخت غذا


,date,title,intro,body,category
3874,چهارشنبه ۲۶ خرداد ۱۴۰۰ - ۰۸:۴۴,قبل از پخت و پز در ظروف مسی و چدنی این مطلب را...,مدیر نظارت بر مواد غذایی، آشامیدنی معاونت غذا ...,به گزارش همشهری‌آنلاین به نقل از ایسنا، دکتر ح...,LifeSkills
3682,سه‌شنبه ۱۷ اسفند ۱۴۰۰ - ۰۶:۴۶,یک اشتباه رایج موقع گرم کردن غذا در مایکروفر |...,وقتی غذا را در مایکروفر گرم می‌کنیم، در بیشتر ...,به گزارش همشهری آنلاین، بسیاری از افراد متوجه ...,LifeSkills
3614,جمعه ۹ مهر ۱۴۰۰ - ۰۰:۴۲,۴ تمرین ذهنی برای خوردن غذاهای سالم‌تر,بر مبنای تحقیقی که اخیرا محققان دانشگاه ییل ان...,به گزارش همشهری آنلاین به نقل از بی‌بی سی، این...,LifeSkills
2544,سه‌شنبه ۷ اردیبهشت ۱۴۰۰ - ۱۰:۴۳,دلیل جذب شدن به بوی غذا چیست؟,پژوهشگران «دانشگاه هاروارد» در بررسی جدید خود ...,به گزارش همشهری آنلاین، ایسنا به نقل از نوروسا...,Science
1202,سه‌شنبه ۱۰ خرداد ۱۴۰۱ - ۱۴:۵۱,وزیر بهداشت: تجویز دارو در ایران ۲ برابر متوسط...,وزیر بهداشت از تجویز دارو برای بیماران در ایرا...,به گزارش همشهری آنلاین، بهرام عین‌اللهی وزیر ب...,Society
3964,دوشنبه ۵ اردیبهشت ۱۴۰۱ - ۰۷:۴۰,کولیره ؛ نانی که به نخ کشیده می‌شود | زنان کُر...,کولیره نوعی نان سنتی است که کردستانی‌ها از گذش...,به گزارش همشهری آنلاین، کلیره نام یک نان در کر...,LifeSkills
3941,سه‌شنبه ۲۹ تیر ۱۴۰۰ - ۰۷:۳۹,ویژگی‌هایی که از شما یک آشپز خوب می‌سازد,ویژگی‌هایی وجود دارد که با داشتن آنها آشپز بهت...,به گزارش همشهری آنلاین به نقل از نمناک، نیاز ب...,LifeSkills
3668,جمعه ۵ شهریور ۱۴۰۰ - ۰۶:۵۶,اگر تنها یک وعده غذا در روز بخورید، چه اتفاقی ...,اگر وقت غذا خوردن سه تا چهار بار در روز ندارید...,به گزارش همشهری آنلاین به نقل از اسپوتنیک، پاو...,LifeSkills
3621,پنجشنبه ۸ اردیبهشت ۱۴۰۱ - ۱۷:۲۴,تصویر حیرت‌انگیز از یک کباب فروش برنده مسابقه ...,عکسی از یک فروشنده غذای خیابانی که در حال پخت ...,به گزارش همشهری آنلاین، عکاس هندی این عکس را د...,LifeSkills
3955,شنبه ۲۱ فروردین ۱۴۰۰ - ۱۵:۲۹,طرز تهیه کیک شکلاتی تابه‌ای خوشمزه بدون فر,اگر دنبال یک دستور پخت کیک تابه‌ای هستید که هم...,به گزارش همشهری آنلاین، درست کردن کیک‌هایی که ...,LifeSkills


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های پیچیده‌تری را مشاهده می‌کنیم.
    </p>
</div>

In [ ]:
for query in COMPLEX_QUERIES:
    print(f"Complex Query: {query}")
    display(TF_IDF_model.predict(query, mini_dataset, k=3))

Complex Query: چه رژیمی برای لاغری مناسب است؟


,date,title,intro,body,category
3618,پنجشنبه ۲۵ شهریور ۱۴۰۰ - ۰۶:۰۶,طرز تهیه اوتمیل | صبحانه‌ای خوشمزه و رژیمی با ...,اوتمیل یا همان جو دو سر یکی از خوراکی‌های رژیم...,به گزارش همشهری آنلاین به نقل از ایرنا، رژیم گ...,LifeSkills
3609,دوشنبه ۱۸ بهمن ۱۴۰۰ - ۰۷:۵۵,ساده‌ترین ترکیب نوشیدنی برای کاهش وزن که معجزه...,یک نوشیدنی که از آب مخلوط با آبِ لیموی تازه ته...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، آب...,LifeSkills
3939,شنبه ۱۹ تیر ۱۴۰۰ - ۱۸:۴۷,۷ غذای کم‌کالری برای افرادی که رژیم لاغری دارند,غذا به ویژه شام رژیمی و کم کالری بسیار تاثیرگذ...,به گزارش همشهری آنلاین، هوای گرم می‌طلبد غذاها...,LifeSkills


<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ٣. مدل مبتنی بر Transformer
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش یک کلاس تعریف می‌کنیم که بتواند کارهایی که نیاز داریم را با استفاده از مدلی بر پایه‌ی ترنسفرمرها انجام دهد. برای این کار از SentenceTransformer در کتاب‌خانه‌ی HuggingFace بهره می‌بریم و مدل ParsBERT که به دست هوش‌واره فراهم شده است را به عنوان مدل زبانی آن استفاده می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در آغاز که یک شی با استفاده از این کلاس ساخته می‌شود یک SentenceTransformer با استفاده از مدل ParsBERT می‌سازیم تا از آن برای به دست آوردن embeddingها استفاده کنیم. سپس در صورتی که GPU در دسترس باشد این مدل را به روی آن منتقل می‌کنیم. علاوه بر این می‌توانیم یک پیش‌پردازشگر هم به متد سازنده بدهیم. این همان پیش‌پردازشگری است که دیتاست اولیه با آن پردازش شده است. ابزار ما از آن برای پیش‌پردازش درخواست‌ها در زمان تشخیص و تبدیل آن‌ها به فرمتی مطابق با متن اولیه پس از پیش‌پردازش استفاده خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در تابع train_embeddings لیستی از متن‌ها یا لیستی از متن‌های شکسته شده به کلمات را دریافت می‌کنیم تا امبدینگ‌های مربوط به آن‌ها را به دست بیاوریم. در حالت دوم، پیش از ادامه‌ی کار کلمات را در یک متن واحد در کنار هم قرار می دهیم. توجه می‌کنیم که داده‌ها پس از پیش‌پردازش در حالت دوم قرار دارند. در نهایت پس از به دست آمدن امبدینگ‌ها، آن‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از به دست آمدن امبدینگ‌ها می‌توانیم آن‌ها را با استفاده از متد save_embeddings در آدرس دلخواهمان به عنوان یک فایل ذخیره کنیم و در دفعات بعدی به جای محاسبه‌ی دوباره، آن‌ها را با استفاده از متد load_embeddings بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        وقتی که امبدینگ‌ها را محاسبه یا بازیابی کردیم، می‌توانیم با استفاده از متد make_index نمایه‌هایی را با استفاده از ابزار Faiss ایجاد کنیم. این ابزار در به دست آوردن موارد مشابه به درخواست‌ها به ما کمک خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در نهایت پس از اینکه تمام مراحل گفته شده را طی کردیم می‌توانیم با استفاده از متد predict یک درخواست را به ابزارمان بدهیم و این ابزار پس از به دست آوردن امبدینگ مربوط به آن درخواست، تعداد دلخواهی از موارد مربوط به آن را با توجه به مدل ترنسفرمر استفاده شده و نمایه‌ی Faiss تولید شده به ما خواهد داد.
    </p>
</div>

In [23]:
class Transformer:

    def __init__(self, preprocessor=None):
        self.model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
        self.preprocessor = preprocessor
        if torch.cuda.is_available():
            self.model = self.model.to(torch.device('cuda'))
        self.embeddings = None
        self.index = None

    def train_embeddings(self, train_dataset: list):
        if type(train_dataset[0]) == list:
            train_dataset = list(map(lambda doc: ' '.join(doc), train_dataset))
        self.embeddings = self.model.encode(train_dataset, show_progress_bar=True)
        self.embeddings = np.array(
            [embedding for embedding in self.embeddings]).astype('float32')

    def make_index(self, dataset: list):
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index = faiss.IndexIDMap(self.index)
        self.index.add_with_ids(self.embeddings, np.arange(len(dataset)).astype('int64'))

    def save_embeddings(self, path='Transformer_model.pickle'):
        with open(path, 'wb') as file:
            pickle.dump(self.embeddings, file)

    def load_embeddings(self, path='Transformer_model.pickle'):
        with open(path, 'rb') as file:
            self.embeddings = pickle.load(file)

    def predict(self, query: str, dataset: pd.DataFrame, k=10):
        if self.preprocessor:
            query = ' '.join(self.preprocessor.preprocess(query))
        vector = self.model.encode(list([query]))
        D, I = self.index.search(np.array(vector).astype('float32'), k=k)
        return dataset.iloc[I.flatten().tolist()]

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train_embeddings(dataset)
        if mode == 'load':
            self.load_embeddings()
        if save:
            self.save_embeddings()
        self.make_index(dataset)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل Transformer ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [ ]:
transformer_model = Transformer(preprocessor=preprocessor)
transformer_model.prepare(preprocessed_texts, 'load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های مشخص شده را مشاهده می‌کنیم.
    </p>
</div>

In [ ]:
for i, query in enumerate(QUERIES):
    print(f"Query {i + 1}: {query}")
    display(transformer_model.predict(query, dataset, k=10))

Query 1: نتیجه توافق برجام


,date,title,intro,body,category
13233,جمعه ۲۰ اسفند ۱۴۰۰ - ۱۷:۵۲,احتمال دستیابی به توافق همزمان با عید نوروز | ...,یک دیپلمات ارشد اروپایی درباره موقعیت مذاکرات ...,به گزارش همشهری‌آنلاین، دیپلمات ارشد اروپایی د...,Politics
9649,جمعه ۲۶ آذر ۱۴۰۰ - ۱۹:۳۳,اظهارات مهم باقری درباره نتایج مذاکرات وین | گ...,مذاکره کننده ارشد ایران در مذاکرات وین پس از پ...,به گزارش هشمهری آنلاین، علی باقری کنی مذاکره ک...,Politics
12693,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۲:۲۳,توییت انتقادی رئیس پیشین صدا و سیما | احتمال ت...,محمدسرافراز در توییتی پیگیری ایران برای «توافق...,به گزارش همشهری‌آنلاین، سرافراز در حساب کاربری...,Politics
456,جمعه ۲۰ فروردین ۱۴۰۰ - ۱۲:۵۰,عکس | آغاز نشست کمیسیون برجام درباره لغو تحریم‌ها,هیجدهمین نشست کمیسیون مشترک برجام نشست خود را ...,,Politics
1030,چهارشنبه ۲۹ اردیبهشت ۱۴۰۰ - ۲۰:۱۵,نماینده اتحادیه اروپا درباره توافق پیرامون برج...,دبیر سیاسی اتحادیه اروپا از رسیدن به توافقی نه...,به گزارش همشهری آنلاین به نقل از ایسنا، صفحه ت...,Politics
1972,سه‌شنبه ۷ اردیبهشت ۱۴۰۰ - ۲۰:۵۷,روایت چین و آلمان از مذاکرات وین درباره برجام|...,مقام‌های چین و آلمان درباره پیشرفت‌های حاصل‌شد...,به گزارش همشهری آنلاین به نقل از فارس، مقام‌ها...,Politics
13506,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۰۱:۴۵,طرفین برجام روشنایی پایان تونل را می‌بینند | م...,مذاکره کننده ارشد چین در مذاکرات وین پس از پای...,به گزارش همشهری آنلاین، ایسنا به نقل از العالم...,Politics
6816,یکشنبه ۱۴ شهریور ۱۴۰۰ - ۲۲:۲۴,یادداشت دیپلمات روس درباره برجام و اهداف مذاکرات,یک دیپلمات روس گفت که هدف از مذاکرات احیای برج...,به گزارش همشهری آنلاین به نق از فارس، مذاکره ک...,Politics
10392,سه‌شنبه ۹ آذر ۱۴۰۰ - ۱۶:۴۶,منبع ایرانی: توافق موقت را نمی‌پذیریم/ اولویت ...,یک منبع ایرانی به شبکه الجزیره گفت که توافق مو...,به گزارش همشهری‌آنلاین به نقل از فارس، یک منبع...,Politics
2278,یکشنبه ۳۰ خرداد ۱۴۰۰ - ۱۴:۰۳,برجام بازمی‌گردد؟ | عراقچی: تقریبا همه اسناد ت...,رئیس تیم ایران در مذاکرات وین گفت: فکر می‌کنم ...,به گزارش همشهری آنلاین به نقل از ایسنا، سید عب...,Politics


Query 2: حمله ارتش روسیه به اوکراین


,date,title,intro,body,category
17768,پنجشنبه ۵ اسفند ۱۴۰۰ - ۲۳:۲۷,تلخ‌ترین تصویر از فرار مردم اوکراین پس از حمله...,از صبح امروز ارتش روسیه به اوکراین حمله کرده و...,,World
17787,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۷:۲۱,تصاویر | نخستین اسرای جنگ اوکراین | دو سربازان...,دو سرباز روسی اسیر شده توسط ارتش اوکراین در جب...,به گزارش همشهری آنلاین به نقل از مشرق، رسانه ه...,World
17670,یکشنبه ۸ اسفند ۱۴۰۰ - ۰۰:۳۳,این نقشه نشان می‌دهد که کی‌یف اکنون در چه وضعی...,نیروهای روسیه کنترل فرودگاه هوستومل در غرب کی‌...,به گزارش همشهری آنلاین، نیروهای روس روز جمعه ب...,World
17659,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۳:۱۶,نقشه مسیر پیشروی ارتش روسیه در اوکراین,همانطور که این نقشه، پیشروی‌های روسیه در شمال ...,به گزارش همشهری آنلاین، این شهر که حدود ۱.۵ می...,World
13628,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۲:۳۴,وحشت اسرائیلی‌ها از ایران پس از حمله روسیه به ...,نشریه صهیونیستی جروزالم پست در تحلیلی به تاثیر...,به گزارش همشهری آنلاین به نقل از ایرنا، در پی ...,Politics
17786,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۷:۲۷,تصاویر | آتش سوزی مشکوک مقابل وزارت دفاع اوکراین,رسانه‌ها با انتشار تصاویری از مقابل ساختمان وز...,به گزارش همشهری آنلاین، حملات ارتش روسیه تا کن...,World
18155,یکشنبه ۷ فروردین ۱۴۰۱ - ۰۸:۵۹,ببینید | حمله موشک‌های «کالیبر» روسیه به انبار...,وزارت دفاع روسیه می‌گوید ارتش این کشور بامداد ...,به گزارش همشهری آنلاین، وزارت دفاع روسیه تصریح...,World
17675,شنبه ۷ اسفند ۱۴۰۰ - ۲۲:۱۸,رئیس‌جمهور چچن: پوتین برای حمله به اوکراین تصم...,رئیس جمهوری چچن با بیان اینکه پوتین تصمیم درست...,به گزارش همشهری آنلاین به نقل از ایسنا، «رمضان...,World
17750,جمعه ۶ اسفند ۱۴۰۰ - ۰۹:۵۷,عکس | چهره خاص بایدن پس از آغاز حمله روسیه به ...,بایدن در سخنانی تحریم‌های جدید علیه روسیه را ا...,منبع: مشرق\n\n\nلحظه به لحظه با آخرین گزارش‌ها...,World
17074,دوشنبه ۱۸ بهمن ۱۴۰۰ - ۰۲:۰۴,ببینید | خط و نشان اتمی روسیه برای اوکراین با ...,روسیه ۲ بمب افکن خود را در نزدیکی اوکراین به پ...,به گزارش همشهری آنلاین به قل از باشگاه خبرنگار...,World


Query 3: افزایش نرخ تورم در کشور


...

,date,title,intro,body,category
59933,سه‌شنبه ۲۸ دی ۱۴۰۰ - ۱۲:۳۵,فیلم‌های جشنواره فجر چهلم را کجا ببینیم؟,جداول نمایش سینماهای مردمی چهلمین جشنواره فیلم...,به گزارش همشهری آنلاین به نقل از روابط عمومی ج...,Culture
64420,یکشنبه ۲۷ تیر ۱۴۰۰ - ۰۱:۰۷,عکس روز | اسپایک لی در اختتامیه جشنواره کن,اسپایک لی، کارگردان آمریکایی و رئیس هیات داورا...,Valéry Hache/AFP/Getty Images,LifeSkills
67260,پنجشنبه ۱۴ بهمن ۱۴۰۰ - ۲۲:۲۸,عکس | لباسی که فرشته حسینی با آن به جشنواره فج...,تصویری از فرشته حسینی و حسین سلیمانی را در چهل...,تصویری جدید از مهدی پاکدل و همسرش در جشنواره ف...,LifeSkills
59598,سه‌شنبه ۳۰ آذر ۱۴۰۰ - ۱۲:۰۷,آخرین مهلت ثبت‌نام مستندسازان در جشنواره فیلم فجر,دبیرخانه چهلمین جشنواره فیلم فجر امروز سه‌شنبه...,به گزارش همشهری آنلاین به نقل از روابط‌عمومی ج...,Culture
60580,پنجشنبه ۱۴ بهمن ۱۴۰۰ - ۲۰:۵۸,عکس | بازیگر بزرگ سینمای ایران در جشنواره فیلم...,در چهارمین روز از برگزاری جشنواره فیلم فجر علی...,به گزارش همشهری‌آنلاین، علی نصیریان از مهمانان...,Culture
60416,جمعه ۲۲ بهمن ۱۴۰۰ - ۱۹:۵۳,تصاویر | چهره‌های سینمایی در اختتامیه چهلمین ج...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، برخی چهره های سینمایی ...,Culture
67237,شنبه ۱۶ بهمن ۱۴۰۰ - ۱۸:۲۰,متفاوت‌ترین عکس مشترک از فرشته حسینی و نوید مح...,تصویری از فرشته حسینی و نوید محمدزاده در جشنوا...,,LifeSkills
60415,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
60426,جمعه ۲۲ بهمن ۱۴۰۰ - ۱۳:۰۰,مجری اختتامیه جشنواره فیلم فجر ۱۴۰۰ کیست؟,مراسم اختتامیه چهلمین جشنواره فیلم فجر به صورت...,به گزارش همشهری آنلاین، چهلمین دوره جشنواره فی...,Culture
60408,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۳:۲۴,پیشنهاد روز: مجله برندگان ادوار جشنواره فیلم ف...,جایزه اصلی جشنواره فیلم فجر سیمرغ بلورین است. ...,به گزارش همشهری آنلاین، مجله برندگان ادوار جشن...,Culture


Query 9: رشد رمزارزها


,date,title,intro,body,category
19906,چهارشنبه ۲۶ خرداد ۱۴۰۰ - ۰۶:۴۰,معروفترین رمزارزهای جهان چگونه به وجود آمدند؟ ...,معروف‌ترین رمزارزهای جهان، از بیت‌کوین گرفته ت...,همشهری- هدا عربشاهی: سال ۲۰۰۸ شخص یا اشخاصی با...,Economy
63438,سه‌شنبه ۲۷ اردیبهشت ۱۴۰۱ - ۰۸:۱۰,تلاش رمزارزها برای فرار از بحران | صندوق پشتیب...,صندوق پشتیبان رمزارز UST اعلام کرد تقریباً تما...,به‌گزارش همشهری آنلاین به نقل از سی‌ان‌بی‌سی‌،...,IT
67292,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۰۶:۵۰,اینفوگرافیک | خودروهای مورد علاقه مردم در سراس...,خودروهای مورد علاقه مردم در سراسر جهان را در ا...,منبع: تسنیم,LifeSkills
23669,دوشنبه ۳ آبان ۱۴۰۰ - ۰۸:۲۸,خطر در کمین دارایی‌های رمزارز کاربران ایرانی‌,شواهد حاکی از آن است که هر لحظه امکان مسدود شد...,همشهری- عمادالدین قاسمی‌پناه: چندی پیش وزارت خ...,Economy
63208,شنبه ۱۶ بهمن ۱۴۰۰ - ۱۲:۴۳,هشدار در مورد پیامک‌های فیشینگ | بر روی آنها ک...,«چانگ‌پنگ ژائو» مدیرعامل صرافی دیجیتال بایننس ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت کوی...,IT
53939,پنجشنبه ۲۰ خرداد ۱۴۰۰ - ۰۹:۲۲,دستگیری ۱۱۰۰ نفر در چین به اتهام پولشویی با ار...,وزارت امنیت عمومی چین اعلام کرد پلیس بیش از ۱۱...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,Science
18781,جمعه ۲۷ فروردین ۱۴۰۰ - ۰۷:۰۰,اینفوگرافیک ا کدام رمزارزها در حال رشد شدید هس...,درصد تغییرات ارزش رمزارزها، سال ۲۰۲۱ را به سال...,به گزارش همشهری آنلاین به نقل از سایت تحلیلی ا...,Economy
54013,سه‌شنبه ۱۱ خرداد ۱۴۰۰ - ۱۲:۴۵,دبی‌کوین، رمزارز خطرناک,این روزها در محافل خبری رمزارزها، نام ارز جدید...,یکی از جدیدترین ارزهای دیجیتال که این روزها در...,Science
19440,چهارشنبه ۱۵ اردیبهشت ۱۴۰۰ - ۱۸:۳۷,هشدار مهم بانک مرکزی به مردم درباره رمزارزها,بانک مرکزی در اطلاعیه‌ای اعلام کرد: طبق مصوبه ...,به گزارش همشهری‌آنلاین، روابط عمومی بانک مرکزی...,Economy
63172,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۸:۴۴,چطور با آیفون در ارز دیجیتال سرمایه‌گذاری کنیم؟,با رشدی که بازار ارزهای دیجیتال در سال ۲۰۲۱ تج...,ارز دیجیتال چیست؟\nارز دیجیتال در اصل یک نوع پ...,IT


Query 10: روش پخت غذا


,date,title,intro,body,category
65959,سه‌شنبه ۱۸ آبان ۱۴۰۰ - ۰۹:۵۸,طرز تهیه لوبیا و گوشت؛ غذای محلی کاشان,برخی غذاهای کاشان بسیار خوشمزه است و افرادی که...,به گزارش همشهری آنلاین به نقل از ایرنا، اگر شم...,LifeSkills
67728,چهارشنبه ۴ اسفند ۱۴۰۰ - ۱۴:۴۴,چی بپزم؟ | طرز تهیه ۶ غذای خوشمزه با بادمجان,طرز تهیه انواع غذا با بادمجان در تمام فصول سال...,به گزارش همشهری آنلاین، بادمجان یکی از صیفی‌جا...,LifeSkills
67608,یکشنبه ۱۵ اسفند ۱۴۰۰ - ۰۹:۵۶,چی بپزیم؟ | طرز تهیه ۶ غذای بدون برنج و آبکی ب...,ناهار چی بپزم؟ این سوالی است که تمام خانم‌ها ه...,به گزارش همشهری آنلاین، ناهار چی بپزم؟ این سوا...,LifeSkills
65868,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۰۷:۳۹,چی بپزیم؟ | طرز تهیه ۷ غذای ساده برای ناهار,اگر شما هم یک خانم خانه‌دار هستید و از همان صب...,به گزارش همشهری آنلاین، اگر شما هم خانه‌دار هس...,LifeSkills
66414,دوشنبه ۱۵ آذر ۱۴۰۰ - ۲۰:۰۸,چی بپزم؟ | طرز تهیه ۵ غذای ساده و راحت برای شا...,معمولا از همان صبح به محض اینکه از خواب بیدار ...,به گزارش همشهری آنلاین، چه خانه‌دار باشید چه ش...,LifeSkills
67685,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۹:۱۸,چی بپزم؟ | طرز تهیه ۵ غذای راحت بدون برنج | غذ...,ناهار چی بپزم؟ این سوالی است که تمام خانم‌ها ه...,به گزارش همشهری آنلاین، برای تمام کدبانوها این...,LifeSkills
65308,شنبه ۱۳ شهریور ۱۴۰۰ - ۱۰:۲۲,طرز تهیه خمیر جادویی؛ مناسب برای پخت انواع پیر...,طرز تهیه خمیر جادویی را بخوانید تا با آن غذاها...,به گزارش همشهری آنلاین به نقل از ایرنا، مطمئنا...,LifeSkills
64859,چهارشنبه ۲۰ مرداد ۱۴۰۰ - ۱۸:۴۵,طرز تهیه نخودآب | این خوراک مقوی و خوشمزه را ف...,نخودآب، نخوداب یا نخود آب یک غذای بسیار مقوی و...,به گزارش همشهری آنلاین به نقل از ایرنا، نخودآب...,LifeSkills
66837,دوشنبه ۱۳ دی ۱۴۰۰ - ۲۳:۰۱,یک اشتباه رایج هنگام پخت ماکارونی از زبان ایتا...,آب‌پز کردن ماکارونی ساده‌ترین تکنیک‌ آشپزی است...,به گزارش همشهری آنلاین، ماکارونی ساده‌ترین و ر...,LifeSkills
64836,جمعه ۲۲ مرداد ۱۴۰۰ - ۱۷:۱۶,طرز تهیه دمپختک | غذایی خوشمزه که در سریع‌ترین...,دمپختک از آن غذاهایی است که با گذشت زمان و تفا...,به گزارش همشهری آنلاین به نقل از ایرنا، دمپختک...,LifeSkills


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های پیچیده‌تری را مشاهده می‌کنیم.
    </p>
</div>

In [25]:
for query in COMPLEX_QUERIES:
    print(f"Complex Query: {query}")
    display(transformer_model.predict(query, dataset, k=3))

Complex Query: چه رژیمی برای لاغری مناسب است؟


,date,title,intro,body,category
66533,یکشنبه ۷ آذر ۱۴۰۰ - ۰۶:۵۶,خواص لوبیا سبز برای لاغری,با گنجاندن لوبیا سبز در رژیم غذایی روزانه خود ...,به گزارش همشهری آنلاین به نقل از خبرگزاری صداو...,LifeSkills
66210,یکشنبه ۲۸ آذر ۱۴۰۰ - ۲۳:۳۵,بهترین رژیم غذایی برای کاهش وزن چیست؟,بسیاری از فرهنگ‌ها و کشورها در سراسر جهان رژیم...,به گزارش همشهری آنلاین به نقل از خبر آنلاین، ب...,LifeSkills
67070,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۰۰:۱۱,بهترین نوشیدنی برای پیشگیری از کبد چرب,یک فوق تخصص گوارش و کبد بزرگسالان درباره بهتری...,به گزارش همشهری آنلاین به نقل از ایرنا، دکتر ش...,LifeSkills


<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ٤. مدل مبتنی بر میانگین وزن‌دار بردارهای تعبیه، مانند FastText
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش، از کتاب‌خانه‌ی fasttext استفاده می‌کنیم و با استفاده از روش skipgram و یک فایل شامل داده‌های موجود اخبار، مدل را به صورت unsupervised به یادگیری وامی‌داریم. پارامترهای استفاده شده‌ی دیگر هم در کد مشخصند؛ به طور مثال، کمینه و بیشینه n را برابر ۲ و ۵ در نظر گرفته‌ایم. پس از انجام یادگیری، میانگین امبدینگ‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم. تمام این کارها با صدا زدن متد prepare با حالت train
        رخ می‌دهند. همچنین می‌توانیم این متد را با حالت load صدا بزنیم تا یک مدل ذخیره شده را از فایل بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        علاوه بر این، می‌توانیم با استفاده از متدهای save_FastText_model و load_FastText_model این مدل و جزئیاتش را ذخیره یا بازیابی کنیم. البته اگر بخواهیم عمل ذخیره را بلافاصله پس از پایان یادگیری انجام دهیم هم می‌توانیم مقدار True را به آرگومان save در متد prepare بدهیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از پایان یادگیری یا بازیابی مدل یادگرفته شده از فایل ذخیره شده، می‌توانیم با استفاده از متد predict به تعداد دلخواه خبر مرتبط به درخواستمان را به دست بیاوریم. برای این کار، مدلمان بردارهای مربوط به درخواست را به دست می‌آورد و با محاسبه‌ی کسینوس‌ها و به دست آوردن میزان شباهت بین درخواست و خبرهای موجود، شبیه‌ترین خبرها را خروجی می‌دهد.
    </p>
</div>

In [19]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        with open('FastText_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), texts))))
        self.model = fasttext.train_unsupervised('FastText_train.txt', self.method, minn=2, maxn=5, wordNgrams=10)
        os.remove('FastText_train.txt')
        self.mean_embed = list(map(lambda doc:
                                   np.mean(list(map(lambda word:
                                                    self.model.get_word_vector(word), doc)), axis=0), texts))
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        if self.preprocessor:
            query = self.preprocessor.preprocess(query)
        if type(query) == str:
            query = query.split()
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train(dataset)
        if mode == 'load':
            self.load_FastText_model()
        if save:
            self.save_FastText_model()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل FastText ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [ ]:
FastText_model = FastText()
FastText_model.prepare(mini_preprocessed_texts, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های مشخص شده را مشاهده می‌کنیم.
    </p>
</div>

In [21]:
for i, query in enumerate(QUERIES):
    print(f"Query {i + 1}: {query}")
    display(FastText_model.predict(query, mini_dataset, k=10))

Query 1: نتیجه توافق برجام


,date,title,intro,body,category
398,یکشنبه ۱۴ شهریور ۱۴۰۰ - ۱۲:۰۸,نماینده تهران در مجلس: ما از مذاکره نمی‌ترسیم ...,نماینده تهران در مجلس گفت: در مجلس نهم مخالفان...,به گزارش همشهری آنلاین به نقل از برنا، اسماعیل...,Politics
133,شنبه ۱۱ دی ۱۴۰۰ - ۰۷:۳۵,مذاکرات مثبت است اما ... | یک خواسته ایران کار...,یک رسانه آلمانی می‌گوید فضای کلی مذاکرات اتمی ...,به گزارش همشهری‌آنلاین به نقل از شفقنا، دپ‌آ د...,Politics
390,سه‌شنبه ۹ آذر ۱۴۰۰ - ۱۴:۴۰,آغاز گفت‌وگوهای کارشناسی درباره لغو تحریم‌ها د...,کارگروه لغو تحریم‌های ظالمانه و غیرقانونی آمری...,به گزارش همشهری‌آنلاین به نقل از ایرنا، این کا...,Politics
344,شنبه ۴ دی ۱۴۰۰ - ۰۷:۴۱,خبر مهم منبع اروپایی از مذاکرات وین | احتمال ت...,یک منبع اروپایی گفت: بین ما و هیات ایرانی نهای...,به گزارش همشهری آنلاین به نقل از ایرنا، ایران ...,Politics
214,دوشنبه ۱۰ آبان ۱۴۰۰ - ۱۴:۵۱,چین: آمریکا باید سیاست اشتباه «فشار حداکثری» ع...,سخنگوی وزارت امور خارجه چین با تأکید بر حمایت ...,به گزارش همشهری‌آنلاین به نقل از فارس، «وانگ و...,Politics
284,شنبه ۲۸ اسفند ۱۴۰۰ - ۱۴:۳۵,آغاز سال ۱۴۰۱ با توافق در وین؟ | ادعای ایندیپن...,یک رسانه‌ انگلیسی به نقل از یکی از طرف‌های مذا...,به گزارش همشهری آنلاین به نقل از فارس، روزنامه...,Politics
67,شنبه ۹ بهمن ۱۴۰۰ - ۱۲:۴۷,اظهارات مقام آمریکایی درباره دلیل توقف مذاکرات...,یک مقام وزارت خارجه آمریکا در پی وقفه چند روزه...,به گزارش همشهری آنلاین به نقل از برنا، یک سخنگ...,Politics
185,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۰۷:۳۰,درخواست مهم گوترش از آمریکا درباره ایران | از ...,دبیرکل سازمان ملل متحد در یک گزارش رسمی از آمر...,به گزارش همشهری آنلاین به نقل از فارس، «آنتونی...,Politics
126,دوشنبه ۲۲ آذر ۱۴۰۰ - ۱۷:۴۹,اقدامات مخرب آمریکا علیه ایران روی میز مذاکرات...,تیم مذاکره‌کننده ایرانی فهرستی از تحریم‌های آم...,به گزارش همشهری آنلاین به نقل از تسنیم، شنیده‌...,Politics
315,چهارشنبه ۳ شهریور ۱۴۰۰ - ۲۲:۱۶,نماینده آمریکا در امور ایران: آماده ازسرگیری م...,رابرت مالی نماینده ویژه آمریکا مدعی شد: اگر ای...,به گزارش همشهری‌آنلاین به نقل از جماران، رابرت...,Politics


Query 2: حمله ارتش روسیه به اوکراین


,date,title,intro,body,category
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
522,سه‌شنبه ۳۰ فروردین ۱۴۰۱ - ۱۶:۲۲,آغاز مرحله جدید عملیات روسیه | موج دوم جنگ در ...,وزیر خارجه روسیه از مرحله جدید عملیات نظامی وی...,به گزارش همشهری آنلاین به نقل از فارس، «سرگئی ...,World
3258,جمعه ۱۳ اسفند ۱۴۰۰ - ۱۹:۰۰,نخستین نبرد هیبرید جهان | اوکراینی‌ها می‌گویند...,مقامات امنیت سایبری اوکراین می‌گویند این کشور ...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,IT
489,جمعه ۵ فروردین ۱۴۰۱ - ۲۳:۳۷,کشته شدن یک فرمانده روسی دیگر در خرسون | آمار ...,ارتش اوکراین ادعا کرد، ژنرال یاکوف رزانتسف از ...,به گزارش همشهری آنلاین، ایرنا به نقل از سی ان ...,World
793,شنبه ۲۱ اسفند ۱۴۰۰ - ۱۷:۲۹,هشدار روسیه: کاروان‌های حامل سلاح برای اوکراین...,یک دیپلمات ارشد روسیه هشدار داده است که مسکو ا...,به گزارش همشهری آنلاین به نقل از گاردین این اظ...,World
586,شنبه ۷ اسفند ۱۴۰۰ - ۲۰:۳۴,ادعای اوکراین؛ شلیک به خودی ناو روس,ستاد مشترک اوکراین امروز، شنبه اعلام کرد، کشتی...,به گزارش همشهری آنلاین‏، ایسنا به نقل از العین...,World
647,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۷:۳۰,ترس شدید ناتو از پیشروی احتمالی روسیه | دبیرکل...,«ینس استولتنبرگ» دبیرکل سازمان پیمان آتلانتیک ...,به گزارش همشهری آنلاین به نقل از تسنیم، دبیرکل...,World


Query 3: افزایش نرخ تورم در کشور


...

,date,title,intro,body,category
2826,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
3145,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۱۵:۵۶,درخشش کلاس رانندگی در آلمان,مرضیه ریاحی از هجدهمین جشنواره جهانی فیلم کوتا...,به گزارش همشهری آنلاین به نقل از روابط عمومی ف...,Culture
2921,جمعه ۲۲ بهمن ۱۴۰۰ - ۱۹:۳۶,اسامی برندگان سیمرغ‌های بلورین جشنواره فیلم فج...,مراسم اختتامیه چهلمین جشنواره فیلم فجر برگزار شد.,به گزارش همشهری آنلاین، آیین اختتامیه چهلمین ج...,Culture
3037,یکشنبه ۳ بهمن ۱۴۰۰ - ۲۰:۲۰,۳ جایزه جشنواره داکا برای زنان سینماگر ایران,جایزه بهترین فیلم زنان فیلمساز، بهترین مستند و...,به گزارش همشهری‌آنلاین به نقل از ایسنا، بیستمی...,Culture
3184,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۱۵:۴۰,بی‌مادر در مسیر جشنواره فیلم فجر,فیلم سینمایی «بی‌مادر» این روزها در حال آماده‌...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Culture
2972,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۶:۳۹,زندگینامه: ساره بیات (۱۳۵۸-),ساره بیات در سال ۱۳۵۸ در مشهد بدنیا آمد.,به گزارش همشهری آنلاین، ساره بیات، زاده ۱۴ مه...,Culture
2940,چهارشنبه ۲۱ مهر ۱۴۰۰ - ۱۶:۰۸,تئاتر فجر ۴۰ قطعاً آثار خارجی خواهد داشت | حضو...,دبیر چهلمین جشنواره تئاتر فجر با تأکید بر اینک...,به گزارش همشهری آنلاین به نقل از مهر، حسین مسا...,Culture
3978,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۰۰:۰۶,عکس | ژست متفاوت صبا سهیلی کنار سینا مهراد در ...,تصاویری از سینا مهراد و خواهرش صبا سهیلی را مش...,به گزارش همشهری آنلاین، در دومین روز برگزاری چ...,LifeSkills
2932,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۲:۰۷,امین حیایی و طناز طباطبایی بهترین بازیگران جشن...,مراسم اختتامیه جشنواره فیلم فجر۱۴۰۰ لحظاتی پیش...,به گزارش همشهری آنلاین، اعضای هیئت داوران بخش...,Culture
3017,چهارشنبه ۲۲ دی ۱۴۰۰ - ۱۵:۳۵,زندگینامه: میترا حجار (۱۳۵۵-),میترا حجار در سال ۱۳۵۵ در شهر مشهد بدنیا آمد. ...,به گزارش همشهری آنلاین، میترا حجار، زادهٔ ۱۴ ب...,Culture


Query 9: رشد رمزارزها


,date,title,intro,body,category
3249,دوشنبه ۱ فروردین ۱۴۰۱ - ۰۹:۵۸,ارز شیبا بخریم یا دوج کوین ؟,اگر در بازار کریپتو کارنسی و رمز ارزها فعالیت ...,این دو رمز ارز امروزه در ردیف محبوب‎ترین رمز ا...,IT
3215,چهارشنبه ۱۱ اسفند ۱۴۰۰ - ۱۲:۰۷,چگونه از ارزهای دیجیتال درآمد کسب کنیم؟,اگر شما برای ورود به بازار کریپتوکارنسی علاقه‌...,اما واقعیت این است که مردم برای کسب درآمد وارد...,IT
3593,یکشنبه ۱۵ اسفند ۱۴۰۰ - ۱۲:۰۲,سرمایه‌گذاری در بورس بهتر است یا ارز دیجیتال ؟,بازار بورس و ارزهای دیجیتال، دو مورد از محبوب‌...,بازار ارزهای دیجیتال به طور قابل توجهی بی‌ثبات...,IT
3202,یکشنبه ۱۷ بهمن ۱۴۰۰ - ۱۱:۲۷,بهترین ارز دیجیتال برای ترید روزانه,بسیاری از سرمایه گذاران ارزهای دیجیتال و تریدر...,برای ورود به ترید روزانه اولین قدم انتخاب ارزه...,IT
3225,دوشنبه ۲۹ فروردین ۱۴۰۱ - ۱۷:۲۵,آموزش کسب درآمد از رمزارز تتر بصورت دلاری,طبق تحقیقات انجام شده، ارز تتر از سال ۲۰۲۰ به ...,در واقع پروژه‌های بلاک چین، روزبه‌روز پیشرفت م...,IT
3351,یکشنبه ۱۷ بهمن ۱۴۰۰ - ۱۷:۲۳,چند رمزارز خوب برای سرمایه‌گذاری در سال ۲۰۲۲,سال ۲۰۲۱ نیز با تمام فرازو نشیب‌هایی که در دنی...,پس از مدتی که بیت کوین نفس تازه کرد، مجدداً شر...,IT
3304,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۴:۵۱,ارز شیبا اینو؛ Shibaسگ شکاری دنیای رمز ارزها د...,اگر به دنبال جواب این سوال هستید که شیبا اینو ...,از عنوان تعجب نکنید! موضوعمان پاسخ به سوال شما...,IT
3552,دوشنبه ۲۶ اردیبهشت ۱۴۰۱ - ۰۹:۴۷,بیت‌کوین آینده‌ای ندارد | عقیده نگران‌کننده مد...,«سام بنکمن-فرید» بنیانگذار صرافی دیجیتال FTX د...,به گزارش همشهری آنلاین و به نقل از روزنامه فای...,IT
3364,چهارشنبه ۲۱ اردیبهشت ۱۴۰۱ - ۱۷:۴۶,شیبا اینو با وجود ضرر همچنان محبوب است | رمزار...,بازار رمزارزها در هفته گذشته دستخوش تغییراتی ش...,به‌ گزارش همشهری آنلاین به نقل از واچر نیوز، ر...,IT
1073,چهارشنبه ۲۲ اردیبهشت ۱۴۰۰ - ۱۵:۳۴,بازار متلاطم پول های دیجیتال,استلار و اتریوم دوشنبه بسیار خوبی را سپری کردن...,همشهری- ساسان شادمان منفرد: دوشنبه گذشته، رمزا...,Economy


Query 10: روش پخت غذا


,date,title,intro,body,category
3901,سه‌شنبه ۲۹ تیر ۱۴۰۰ - ۲۳:۰۵,طرز تهیه ۵ غذای شمالی,غذاهای شمالی بسیار خوشمزه و پرطرفدارند. در بیش...,به گزارش همشهری‌آنلاین، از جمله غذاهای محلی شم...,LifeSkills
3954,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۱۸:۲۴,چی بپزم؟ | طرز تهیه لوبیا پلو با گوشت چرخ‌ کرد...,لوبیاپلو به شیوه‌های مختلفی با گوشت‌های متفاوت...,به گزارش همشهری آنلاین، لوبیا پلو از جمله پلوه...,LifeSkills
3724,چهارشنبه ۱۱ اسفند ۱۴۰۰ - ۱۷:۴۷,۱۰ اشتباه رایجی که هنگام درست کردن جوجه کباب ا...,جوجه کباب یکی از رایج‌ترین و پرطرفدارترین غذاه...,به گزارش همشهری آنلاین، یکی از اشتباهاتی که اغ...,LifeSkills
3973,سه‌شنبه ۲۲ تیر ۱۴۰۰ - ۱۶:۳۰,طرز تهیه هویج پلو | فوت و فن درست کردن این غذا...,هویج ‌پلو از جمله پلوهای مخلوط قدیمی است که مع...,به گزارش همشهری آنلاین - صدیقه کیان منش‌راد: ه...,LifeSkills
3837,دوشنبه ۳ خرداد ۱۴۰۰ - ۱۶:۲۵,طرز تهیه عدس پلو | نکاتی برای خوشمزه شدن این غ...,عدس‌پلو معمولا از آن غذاهایی است که برای لذیذ ...,به گزارش همشهری آنلاین، عدس پلو در میان پلوهای...,LifeSkills
3730,پنجشنبه ۳۱ تیر ۱۴۰۰ - ۰۸:۴۰,طرز تهیه کباب تابه‌ای | به ۲ شیوه ساده کباب دی...,۲ طرز تهیه ساده کباب تابه‌ای زعفرانی آبدار مجل...,به گزارش همشهری آنلاین به نقل از اکالا، تابه‌ا...,LifeSkills
3682,سه‌شنبه ۱۷ اسفند ۱۴۰۰ - ۰۶:۴۶,یک اشتباه رایج موقع گرم کردن غذا در مایکروفر |...,وقتی غذا را در مایکروفر گرم می‌کنیم، در بیشتر ...,به گزارش همشهری آنلاین، بسیاری از افراد متوجه ...,LifeSkills
3897,چهارشنبه ۳ آذر ۱۴۰۰ - ۰۶:۰۰,طرز تهیه آش دوغ اردبیلی | فوت و فن درست کردن ا...,آش دوغ یکی از آش‌های خوشمزه و معروف اردبیل است...,به گزارش همشهری آنلاین به نقل از خبرگزاری صداو...,LifeSkills
3822,سه‌شنبه ۹ شهریور ۱۴۰۰ - ۲۳:۰۱,چطور در خانه ذرت مکزیکی رستورانی درست کنیم؟,این مطلب را بخوانید و به‌راحتی در خانه به شیوه...,به گزارش همشهری آنلاین، ذرت مکزیکی یکی از انوا...,LifeSkills
3939,شنبه ۱۹ تیر ۱۴۰۰ - ۱۸:۴۷,۷ غذای کم‌کالری برای افرادی که رژیم لاغری دارند,غذا به ویژه شام رژیمی و کم کالری بسیار تاثیرگذ...,به گزارش همشهری آنلاین، هوای گرم می‌طلبد غذاها...,LifeSkills


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در نهایت، سرچ بر اساس عنوان خبر را پیاده سازی می‌کنیم. این سرچ نیازی به استفاده از مدل‌های زبانی ندارد و صرفاً مانند یک فیلتر عمل می‌کند.
    </p>
</div>

In [16]:
CATEGORIES_INV = {v: k for k, v in CATEGORIES.items()}

def display_news_filtered_by_category(category_per, k):
    filtered_news = []
    for _, data in dataset.iterrows():
        if data['category'] == CATEGORIES_INV[category_per]:
            filtered_news.append(data)
    np.random.shuffle(filtered_news)
    display(pd.DataFrame(filtered_news[:k]))

for category_per in CATEGORIES.values():
    print(f'خبرهای {category_per}')
    display_news_filtered_by_category(category_per, k=10)

خبرهای سیاسی


,date,title,intro,body,category
12440,پنجشنبه ۱۴ بهمن ۱۴۰۰ - ۰۸:۲۷,از بخشش آرارات تا شوهر دادن بحرین! | ۷منطقه ر...,دوره تقریبا ۵ دهه‌ای حاکمیت پهلوی پدر و پسر بر...,به گزارش همشهری آنلاین روزنامه فرهیختگان نوشت:...,Politics
9627,شنبه ۲۷ آذر ۱۴۰۰ - ۱۲:۰۶,اعلام ۲ تاریخ احتمالی برای ازسرگیری مذاکرات وی...,نماینده روسیه در مذاکرات وین ۲ تاریخ احتمالی ب...,به گزارش همشهری‌آنلاین، «میخائیل اولیانوف »، ن...,Politics
4075,یکشنبه ۲۰ تیر ۱۴۰۰ - ۲۱:۱۷,عکس | سفیر جدید کره جنوبی رونوشت استوارنامه خو...,سفیر جدید کره جنوبی در تهران امروز در آغاز مأم...,به گزارش همشهری‌آنلاین به نقل از فارس، یون کان...,Politics
4386,یکشنبه ۱۳ تیر ۱۴۰۰ - ۰۶:۳۰,عکس | صاعقه موشکی ایران بر سر تجهیزات نظامی دش...,راکت فجر ۵ سی، از چهار بالچه هدایت‌شونده برخور...,به گزارش همشهری آنلاین به نقل از میزان، ارتش ج...,Politics
12860,یکشنبه ۳ بهمن ۱۴۰۰ - ۱۱:۲۸,جزئیات پرونده قتل یک شهردار در رستوران | گلوله...,رئیس سازمان قضایی نیروهای مسلح کل کشور گفت: از...,به گزارش همشهری آنلاین به نقل از ایسنا، حجت ال...,Politics
9810,دوشنبه ۲۲ آذر ۱۴۰۰ - ۱۴:۴۳,آخرین وضعیت طرح پر سر و صدای صیانت از فضای مجا...,دبیر کمیسیون مشترک حمایت از حقوق کاربران در فض...,به گزارش همشهری‌آنلاین به نقل از ایسنا، مهردا...,Politics
12985,یکشنبه ۲۹ اسفند ۱۴۰۰ - ۱۴:۱۶,پیام نوروزی پر از کنایه حسن روحانی | حالا که ا...,رئیس دولت‌های یازدهم و دوازدهم در پیامی خطاب ب...,به گزارش همشهری آنلاین به نقل از ایسنا، متن پی...,Politics
3549,یکشنبه ۲ خرداد ۱۴۰۰ - ۲۲:۳۹,یک کاندیدای نظامی خبر انصراف خود را تکذیب کرد,محمد حسن نامی وزیر پیشین ارتباطات و رییس سابق ...,به گزارش همشهری آنلاین به نقل از جماران؛ محسن ...,Politics
7467,چهارشنبه ۲۸ مهر ۱۴۰۰ - ۱۵:۰۵,زندگینامه: بهرام عین‌اللهی (۱۳۳۷-),بهرام عین‌اللهی در سال ۱۳۳۷ در شهر سراب متولد ...,به گزارش همشهری آنلاین، بهرام عین‌اللهی (زادهٔ...,Politics
8570,چهارشنبه ۲۶ آبان ۱۴۰۰ - ۲۱:۱۸,تماس تلفنی نخست وزیر عراق با رئیسی | رئیس جمهو...,سید ابراهیم رئیسی در تماس تلفنی «مصطفی الکاظمی...,به گزارش همشهری آنلاین،‌رئیس جمهور خاطرنشان کر...,Politics


خبرهای جهانی


,date,title,intro,body,category
16879,چهارشنبه ۱۵ دی ۱۴۰۰ - ۰۹:۲۱,اسیرمان به شهادت برسد جنگ با اسرائیل را شروع م...,یکی از رهبران جنبش جهاد اسلامی روز دوشنبه تهدی...,به گزارش همشهری آنلاین، روزنامه کیهان نوشت: «ه...,World
18686,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۴:۳۹,سوئد در برابر شروط اردوغان برای پیوستن به ناتو...,نخست وزیر سوئد اعلام کرد که کشورش از مبارزه با...,به گزارش همشهری‌آنلاین، ایسنا به نقل از روسیا ...,World
18360,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۶:۴۸,تاثیر بحران اوکراین بر تغذیه دنیا | بانک جهانی...,عملیات روسیه در اوکراین به‌طور قابل‌توجهی بر ق...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World
18368,دوشنبه ۱۹ اردیبهشت ۱۴۰۱ - ۱۸:۳۴,برخورد شدید کره‌شمالی با پوشیدن شلوار جین تنگ،...,رسانه موسوم به رادیو آسیای آزاد با استناد به م...,به گزارش همشهری‌آنلاین به نقل از ایسنا، این خب...,World
16474,جمعه ۱۲ شهریور ۱۴۰۰ - ۱۴:۳۷,ابتکار جالب مردم لندن برای اعتراض به نحوه مدیر...,داوطلبان لندنی با نقاشی هزاران قلب قرمز در برا...,به گزارش همشهری آنلاین به نقل از ان‌پی‌آر، لند...,World
17679,شنبه ۷ اسفند ۱۴۰۰ - ۲۰:۳۴,ادعای اوکراین؛ شلیک به خودی ناو روس,ستاد مشترک اوکراین امروز، شنبه اعلام کرد، کشتی...,به گزارش همشهری آنلاین‏، ایسنا به نقل از العین...,World
16358,سه‌شنبه ۱۹ مرداد ۱۴۰۰ - ۱۱:۱۰,بحران بنزین در لبنان سه کشته داد | برق یک ساعت...,تشدید بحران سوخت در لبنان باعث خشونت شد و سه ک...,به گزارش همشهری آنلاین به نقل از الجزیره، لبنا...,World
16469,دوشنبه ۱۵ شهریور ۱۴۰۰ - ۱۵:۴۵,پسر قذافی پس از ۷ سال آزاد شد,ساعدی قذافی، پسر معمر قذافی، دیکتاتور سابق لیب...,به گزارش همشهری آنلاین به نقل از فرانس۲۴، لیبی...,World
18128,سه‌شنبه ۹ فروردین ۱۴۰۱ - ۲۲:۴۰,واکنش رئیس‌ جمهوری آمریکا به نتایج مذاکرات روس...,جو بایدن، رئیس‌جمهوری ایالات متحده، نسبت به مذ...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,World
16584,شنبه ۲۹ آبان ۱۴۰۰ - ۱۴:۴۴,آمریکا نگران واکنش عمومی به برخورد نژادپرستانه...,حکم دادگاه کایل رتینهاوس اعلام شد؛ دادگاه او ر...,به گزارش همشهری آنلاین، حکم دادگاه کایل رتینها...,World


خبرهای اقتصادی


...

,date,title,intro,body,category
61230,چهارشنبه ۱۱ اسفند ۱۴۰۰ - ۱۳:۵۹,رکورد جدید اولین قسمت فصل ۶ سریال پیکی بلایندر...,یکشنبه شب اولین قسمت از فصل ششم و پایانی «پیکی...,به گزارش همشهری آنلاین به نقل از سینمادیلی، سر...,Culture
61371,سه‌شنبه ۳ اسفند ۱۴۰۰ - ۲۱:۰۷,آمریکایی‌ها جنایت بی‌دقت شهرام مکری را روی پرد...,«جنایت بی‌دقت» در کنار ۳ فیلم بلند دیگر از شهر...,به گزارش همشهری آنلاین به نقل از ایسنا، «ماهی ...,Culture
61505,دوشنبه ۲۹ فروردین ۱۴۰۱ - ۱۰:۵۳,واکنش معاون امور هنری وزیر به شایعه لغو کنسرت‌ها,معاون امور هنری وزارت فرهنگ و ارشاد اسلامی با ...,به گزارش همشهری‌آنلاین به نقل از ایرنا، شامگاه...,Culture
57113,یکشنبه ۱۲ اردیبهشت ۱۴۰۰ - ۱۶:۱۹,اسب سفید بالدار خلاقانه‌ترین فیلم جشنواره کانا...,فیلم کوتاه «اسب سفید بالدار» جایزه خلاقانه‌تری...,به گزارش همشهری آنلاین، مهر به نقل از روابط عم...,Culture
59486,یکشنبه ۹ آبان ۱۴۰۰ - ۱۵:۲۶,آغاز حضوری جشنواره فیلم توکیو,در حالی که بیشتر مرزهای ژاپن به سوی مسافران خا...,به گزار ایسنا، ‌سی و چهارمین جشنواره بین‌الملل...,Culture
58122,دوشنبه ۱۸ مرداد ۱۴۰۰ - ۱۸:۰۰,کار جوخه خودکشی نگرفت | فروش ناامیدکننده در او...,«جوخه خودکشی» فیلمی از جیمز گان با فروش ۲۶.۵ م...,به گزارش همشهری آنلاین، مهر به نقل از ورایتی ن...,Culture
56898,جمعه ۶ فروردین ۱۴۰۰ - ۰۶:۵۸,موسیقی سنتی ما مثل تخت جمشید و سی و سه پل است ...,۲۷سال است که می‌خواند؛ سنتی و ایرانی. صدایش با...,چه زمانی که ردیف‌های آوازی بنان را در همراهی ب...,Culture
62362,دوشنبه ۱۹ اردیبهشت ۱۴۰۱ - ۱۲:۴۹,رمزگشایی از هیجان‌انگیزترین فیلم دنیای ابرقهرم...,دکتر استرنج در مولتی‌ورس جنون (Doctor Strange ...,به گزارش همشهری آنلاین، از سال ۲۰۱۹ که نام دنب...,Culture
62884,دوشنبه ۹ خرداد ۱۴۰۱ - ۰۹:۰۷,نقد صنعت زیبایی، زندگی مرفه در فیلم برنده نخل ...,فیلم مثلث غم از روبن اوستلوند، کارگردان سوئدی ...,به گزارش همشهری آنلاین، پنج سال پس از ربودن نخ...,Culture
58441,چهارشنبه ۲۴ شهریور ۱۴۰۰ - ۱۷:۵۲,سایه دینامیت بر گیشه سینماها | درخت گردو در رد...,برای دومین هفته متوالی ۵ فیلم پرفروش سینمای ای...,به گزارش همشهری آنلاین به نقل از ایرنا، تازه‌ت...,Culture


خبرهای فناوری اطلاعات


,date,title,intro,body,category
63129,یکشنبه ۱۹ دی ۱۴۰۰ - ۱۷:۰۹,اسکنر سه‌بعدی سونی، دریچه‌ای برای ورود به متاورس,شرکت سونی همچنان پیگیر تولید اسکنر سه‌بعدی خود...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
63473,پنجشنبه ۱۵ اردیبهشت ۱۴۰۱ - ۱۴:۱۷,نگرانی بیل‌گیتس از خرید توییتر توسط ایلان ماسک...,بیل گیتس برای مقابله با بی‌برنامگی ایلان ماسک ...,به گزارش همشهری ‌آنلاین و به نقل از سی‌ان‌بی‌س...,IT
63081,پنجشنبه ۳۰ دی ۱۴۰۰ - ۱۸:۰۶,کسب درآمد از تولید محتوا در اینستاگرام | قابلی...,در ماه ژوئن ۲۰۲۰، فیسبوک «اشتراک‌» را راه‌اندا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
63312,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۶:۵۶,آزمایش سقوط برای سری گالکسی اس ۲۲ | آیفون بادو...,سری گالکسی اس ۲۲ سامسونگ به تازگی مورد آزمایش ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
63385,پنجشنبه ۱۸ فروردین ۱۴۰۱ - ۲۱:۱۵,زاک باکس، ارز دیجیتال فیسبوک خواهد بود؟ | برنا...,بر اساس گزارش جدید فایننشال تایمز، متا ممکن اس...,به گزارش همشهری آنلاین، طبق گزارش‌ها، شرکت ماد...,IT
63514,جمعه ۲ اردیبهشت ۱۴۰۱ - ۱۹:۲۹,درخواست ویژه جمهوری‌خواهان کنگره از توییتر | م...,گروهی متشکل از ۱۸ جمهوری‌خواه مجلس نمایندگان آ...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT
63132,جمعه ۱۷ دی ۱۴۰۰ - ۲۰:۱۵,پرچم‌داران سونی به اندورید ۱۲ مجهز می‌شوند,سونی تاره‌ترین شرکتی محسوب می‌شود که پرچم‌دارا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
63366,چهارشنبه ۲۴ فروردین ۱۴۰۱ - ۱۸:۳۴,چین علیه NFT | توکن‌های غیرمثلی قانومند می‌شود,چین امروز (چهارشنبه) درباره خطرهای مالی مرتبط ...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT
63454,پنجشنبه ۲۲ اردیبهشت ۱۴۰۱ - ۱۰:۱۷,متاورس مریخ ساخته می‌شود | همکاری ناسا با شرکت...,آژانس فضایی آمریکا (ناسا) با شرکت بازی‌سازی اپ...,به‌گزارش همشهری آنلاین و به نقل از واچر نیوز، ...,IT
63384,پنجشنبه ۱۸ فروردین ۱۴۰۱ - ۲۲:۱۷,ویندوز ۳.۱، ۳۰ ساله شد!,سیستم‌عامل ویندوز ۳.۱ شرکت مایکروسافت، ۳۰ ساله...,به گزارش همشهری آنلاین، ویندوز ۳.۱ به روزرسانی...,IT


خبرهای مهارت‌های زندگی


,date,title,intro,body,category
65898,یکشنبه ۲۳ آبان ۱۴۰۰ - ۲۳:۱۶,۴ فایده عجیبی که هوای سرد برای بدن دارد,سرمای هوا می‌تواند باعث خشکی پوست شده و خطر اب...,به گزارش همشهری آنلاین به نقل از صداوسیما، برا...,LifeSkills
67682,دوشنبه ۹ اسفند ۱۴۰۰ - ۰۱:۵۱,۱۰ دمنوش‌ گیاهی موثر برای پاکسازی ریه‌ها,آلودگی هوا یکی از معضلات جدی عصر حاضر محسوب می...,به گزارش همشهری آنلاین به نقل از فارس، آلودگی ...,LifeSkills
66003,شنبه ۱۵ آبان ۱۴۰۰ - ۰۸:۱۳,۵ راهکار طبیعی برای کنترل قند خون,راهکارهای طبیعی وجود دارند که به شما کمک می‌کن...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,LifeSkills
64427,شنبه ۲۶ تیر ۱۴۰۰ - ۱۲:۱۹,نوشیدن چای در این مواقع خطرناک است,مصرف هر گونه مواد خوراکی اگر به اندازه کافی و ...,به گزارش همشهری آنلاین به نقل از صدا و سیما، چ...,LifeSkills
64705,پنجشنبه ۳ تیر ۱۴۰۰ - ۰۷:۴۱,۵ خوراکی سیاه که باید در رژیم غذایی خود داشته ...,همه ما در مورد اهمیت خوردن سبزیجات برگ‌سبز و م...,به گزارش همشهری آنلاین به نقل از انتخاب، خوراک...,LifeSkills
66970,شنبه ۴ دی ۱۴۰۰ - ۲۱:۴۶,این گیاه باعث افزایش رشد مو می‌شود | نحوه استف...,گیاه بابا آدم موجب تقویت فولیکول‌ها در نتیجه ا...,به گزارش همشهری آنلاین به نقل از مشرق، خاصیت ب...,LifeSkills
67761,دوشنبه ۲ اسفند ۱۴۰۰ - ۰۷:۲۶,تصاویر | انتقال بیماران با لاکچری‌ترین آمبولان...,یک شرکت خدمات آمبولانس و انتقال بیمار در دوبی ...,به گزارش همشهری آنلاین، یک شرکت خدمات آمبولانس...,LifeSkills
65128,یکشنبه ۲۸ شهریور ۱۴۰۰ - ۰۷:۱۹,این نوشیدنی جادویی فشار خون بالا را درمان می‌کند,دانشمندان پاکستانی به تاثیر خارق‌العاده مصرف ی...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,LifeSkills
67165,پنجشنبه ۲۱ بهمن ۱۴۰۰ - ۱۱:۲۴,عکس | اینجا روی مرده‌های خود خاک نمی‌ریختند | ...,کنگان از شهرستان‌های بوشهر است و در جنوب‌غربی ...,به گزارش همشهری آنلاین، بوشهر طولانی‌ترین مرز ...,LifeSkills
66739,سه‌شنبه ۲۱ دی ۱۴۰۰ - ۰۱:۴۶,آیا درد مفصل‌تان به علت آرتروز است؟,آیا زانوی‌تان گاه‌به‌گاه درد می‌گیرد یا انگشتا...,«آرتروز» یا «استئوآرتریت» شایع‌ترین شکل آرتریت...,LifeSkills


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        از این که وقت ارزشمند خود را برای بررسی این نوتبوک صرف کرده‌اید، صمیمانه سپاسگزاریم. (:
    </p>
</div>